# Introduction

The following notebook contains all the codes, graphs and simulations written in Julia version 1.9.0.

This notebook is part of the master thesis "Modeling and dynamical analysis of cortical network activity in semantic priming". The notebook focuses on simulating the activity of a target item encoded in semantic memory. More specifically, a study of the dynamics of the network made of a population of **excitatory** (E) neurons coding for the target item and of a population of **inhibitory** (I) neurons regulating the network is performed. The whole network with these two populations forms what is called a *Wilson-Cowan model* (WC).

Moreover, the master thesis is based on the paper from N. Brunel and F. Lavigne (2009) "Semantic Priming in a Cortical Network Model" (Journal of Cognitive Neuroscience, Vol 21, Issue 12, pp. 2300-2319).
In the following, two methods will be used:

 - A first approach that will use the equations taken directly from B&L's paper. The equation will be adapted to a one-dimensional version of course. This method will be called "Method 1" and all the parameter values for this approach will be extracted from this paper as well.
 
 - A second approach that will use more standard equations of the WC model, especially regarding the transfer function that is used, that is a sigmoidal function. These equations and the corresponding parameters are inspired from the Neuromatch Academy Computational Neuroscience 2021 Course that can be found on this [website](https://compneuro.neuromatch.io/tutorials/W2D4_DynamicNetworks/student/W2D4_Tutorial1.html) (last accessed 01/08/23).

# Coding part
## 0.1 Settings

In [ ]:
#= =====================================
# Install useful packages if needed
using Pkg

Pkg.add("Parameters")

# Plotting packages
Pkg.add("Plots")
Pkg.add("GLMakie")
Pkg.add("LaTeXStrings")

# Mathematical operation packages
Pkg.add("LinearAlgebra")
Pkg.add("QuadGK")
Pkg.add("Statistics")
Pkg.add("OrdinaryDiffEq")
Pkg.add("NLsolve")
Pkg.add("Printf")
===================================== =#

In [ ]:
# Importing useful packages
using Parameters

# Plotting packages
using Plots, LaTeXStrings
# Avoid name conflicts with GLMakie later
import Plots.plot as plot
import Plots.plot! as plot!
import Plots.scatter as scatter
import Plots.scatter! as scatter!
import Plots.contour as contour
import Plots.contour! as contour!
import Plots.text as text
import Plots.heatmap as heatmap
import Plots.heatmap! as heatmap!

using Plots.PlotMeasures
import Plots.PlotMeasures.w as width
import Plots.PlotMeasures.px as px

colors = [:CornflowerBlue,:DarkOrange,:Magenta,:DarkGray,:Firebrick,
          :LightSeaGreen,:violet,:Gold,:MediumBlue,:DeepPink,:BurlyWood,:black,
          :Red,:Cyan,:teal,:Coral,:MediumTurquoise, :MediumVioletRed,:Indigo,
          :Peru,:RoyalBlue,:PaleVioletRed, :SeaGreen,:Magenta,:Marroon,
          :Crimson, :IndianRed, :DarkCyan, :Darkviolet]

# Mathematical operation packages
using LinearAlgebra, QuadGK, Statistics, NLsolve
using OrdinaryDiffEq

# Debugging (mainly)
using Printf

## 0.2 Helper functions

In [ ]:
@with_kw struct default_pars_BL_target
    #= ========================================================================
        Structure used to get default parameters for the Method 1
        whenever needed.
    ======================================================================== =#
    
    # Excitatory parameters
    tau_E = 10.         # Time scale of the E population rate dynamics [msec]
    p = 1               # Number of selective populations. Here, only the
                        # excitatory target population.
    
    # Synaptic connection strengths/efficacies
    JE = 3.             # Average excitatory synaptic strength
    JS = 3.65           # Strength of synaptic potentiation
    JI = JE             # Synaptic efficacy of global inhibition
    J1 = JE + JS        # Intrapopulation efficacy
    w = 0.02            # 1-population adapted recurrent weight. 
                        # Should be < 1/(dPhi(I_(w=0))) for a stable background
                        # state at r_spont rate (for r_spont = 5, wmax = 0.04)
    
    # External inputs
    I_sel_E = 0.15      # Selective external input current [mA]. Same for all.
    Iapp = t->I_sel_E   # Applied current (time course)
    I_ext_E = -0.2811   # Nonselective external input current [mA]. 
                        # Set to obtain a spontaneous activity of 5 Hz
                        # with w=0.02.
                        # Chosen s.t. I_ext = Phi^-1(r_spont) - w*r_spont
    
    # Rate contstants
    r_spont = 5.        # Spontaneous rate activity [Hz].
    #threshold = 27.     # Threshold for reaction time [Hz].
    
    # Simulation parameters
    T = 50.             # Total duration of simulation [msec]
    dt = .5             # Simulation time step [ms]
    rE_init = 3.        # Initial value of target (or E) rate [Hz]

    # Vector of discretized time points [msec]
    range_t = 0:dt:T-dt
    
end

In [ ]:
@with_kw struct default_pars_generic_target
    #= ========================================================================
        Structure used to get default parameters for Method 2
        whenever needed.
    ======================================================================== =#
    
    # Excitatory parameters
    tau_E = 1.            # Time scale of the E population [msec]
    a_E = 1.2             # Gain/Slope of the E population f-I curve
    theta_E = 2.8         # Threshold of the E population f-I curve

    # Connection strength
    wEE = 9.              # E to E recurrent connection strength

    # External input
    I_ext_E = 0.          # External input current.
    Iapp = t->I_ext_E     # Applied current (time course).
    
    # Rate contstants
    r_spont = 0.          # Spontaneous rate activity [Hz].
    #threshold = theta_E   # Threshold for reaction time [Hz].

    # Simulation parameters
    T = 20.               # Total duration of simulation [msec]
    dt = .1               # Simulation time step [msec]
    rE_init = 0.2         # Initial value of target (or E) rate (or activity)
    
    # Vector of discretized time points [msec]
    range_t = 0:dt:T-dt
    
end

In [ ]:
# Simple test
pars = default_pars_BL_target()
print(pars)

pars = default_pars_generic_target(T = 50.)
print(pars)

In [ ]:
function plot_fI(x, f, xtick_step=1, ytick_step=1, lab=:none,
                                                    colour=:DarkBlue)
    #= ==============================================================
        Function for plotting the f-I curve
    
    Args:
    x    : The input current values. Vector of float.
    f    : The firing rate (or activity values) [Hz]
    
    xtick_step, ytick_step : Step for axes convenience
    lab  : Label of plotted f
    
    Returns:
    Display the plot
    =============================================================== =# 
    plt = plot(x, f, c=colour, label=lab, linewidth=2)
    plt! = plot!(xlabel=L"$\mathrm{Synaptic \ input \ current}$",
               ylabel=L"$\mathrm{Average \ firing \ rate \ [Hz]}$",
               guidefontsize=13, xticks=x[1]:xtick_step:x[end],
               yticks = 0:ytick_step:f[end]+ytick_step,
               xlims=[x[1], x[end]], thickness_scaling=1.2)
    
    #display(plt)
end

## 1 Static current-to-rate (or $f - I$ curve) transfer functions
### 1.1 Excitatory population

#### 1.1.1 Method 1 (using equations from B&L paper directly)
The following transfer function represents an activation function for a population of quadratic integrate-and-fire neurons in presence of Gaussian White Noise, that is random noise. This transfer function was obtained analytically by Brunel and Latham ("Firing Rate of the Noisy Quadratic Integrate-and-Fire Neuron," in Neural Computation, vol. 15, no. 10, pp. 2281-2306, 1 Oct. 2003, DOI: [10.1162/089976603322362365](https://doi.org/10.1162/089976603322362365)). Brunel and Latham state that this $f - I$ curve is expected to be qualitatively similar to that of cortical **excitatory** neurons. 

In other words, this transfer function indicates how a population of E neurons will integrate its inputs in order to initiate or not an electrical response to these inputs. 

In [ ]:
function Phi(x, σ=0.5, τ_memb=10.)
    #= ===============================================================
        Population transfer (or activation) function from B&L paper
    
    Arg:
    x      : The synaptic input current that the population receives 
             (Float64).
    
    σ      : Standard deviation (or square root of the variance) of
             the fluctuations (or noise).
             Note that here, a Gaussian White Noise (GWN) is used.
    
    τ_memb : Time constant of the cell membrane [msec] !.

    
    Returns:
    phi      : The population activation response Phi(x) (Float64) for
               input current x [Hz].
    
    N.B.: If x < -16. mA then the integrand cannot be represented
          numerically because its value is too large. Hence, Phi
          cannot be represented as well. However, since the value of
          Phi theoretically exists and is equal to zero, Phi can be
          assigned the value 0.0 for such values of x.
    =============================================================== =#
    if x >= -16.
        f(z) = exp(-x*z^2 - σ^4*z^6/48)
        integral, error = quadgk(f, -Inf, Inf, rtol=1e-8)

        # Scaling factor 1000 to get Hz
        phi = 1000/(sqrt(pi) * τ_memb * integral)
    else
        phi = 0.0
    end
    
    return phi
end

In [ ]:
# Plot of the f-I curve
gr(guidefontsize=13, labelfontsize=15, legendfontsize=10, grid=false)

x = -2.:0.1:5
phi = Phi.(x)

plot_fI(x, phi, 1.0, 10., :none, :black)

#### 1.1.1.1 Understanding the transfer function's behaviour

In [ ]:
function plot_integrand(x, z, σ=0.5)
    #= ===============================================================
        Plot the integrand exp(-xz^2 - σ^4*z^6/48) to observe its
        shape and its domain of definition
    
    Args:
    x      : Total input current (1D array of float)
    z      : Independent variable (1D array of float)
    σ      : Standard variation of the fluctuation
    
    Returns:
    Plot of the integrand for different values of I
    =============================================================== =#
    ln_x = length(x)
    p = plot()
    
    for i in 1:ln_x
        f = exp.(-x[i] .* z.^2 .- σ^4*z.^6/48)
        
        plot!(z, f, c=colors[i], label=latexstring("$(x[i])"), lw=2)
    end
    plot!(xlabel=L"z", ylabel=L"\exp(-xz^2-\frac{\sigma^4z^6}{48})",
          legendtitle=L"x", guidefontsize=13, thickness_scaling=1.2)
    display(p)
end

In [ ]:
# Examples for I >= 0.0
gr(guidefontsize=13, labelfontsize=15, legendfontsize=10, grid=false)

x = 0.:1.0:5
z = -5.0:0.1:5.0

plot_integrand(x,z)

In [ ]:
# Examples for I < 0.0
gr(guidefontsize=13, labelfontsize=15, legendfontsize=10, grid=false)

x=-0.1:-0.1:-1.
z=-20:0.1:20

plot_integrand(x,z)

In [ ]:
# Display the case where integrand is not representable numerically
gr(guidefontsize=13, labelfontsize=15, legendfontsize=10, grid=false)

x=-16.5
z=-20:0.1:20

plot_integrand(x,z)

#### 1.1.2 Method 2 (using more generic equations)

The interpretation of the transfer function stays the same whatever the type of function that is used, i.e. the transfer function simply states how a population of neurons integrates its inputs.

The following transfer function is more generic due to its sigmoid shape that can be parametrized as the user wants. Moreover, for a same global shape, the exact output value of the function at different input values could be different due to the different parameter values that could be used. Hence, it is possible that the transfer functions for the E population and for the I population have the same global shape but not the same output values. 

In [ ]:
function F(x, α, θ)
    #= ===============================================================
        Population activation function.
    
    Args:
    x (float)    : The population input current
    α (float)    : The gain of the function (> 0)
    θ (float)    : The threshold of the function (> 0)
    
    Returns:
    f (float)    : The population activation response F(x) for
                   input x
    =============================================================== =#
    # Define the sigmoidal transfer function f = F(x)
    f = (1 + exp(-α * (x - θ)))^(-1) - (1 + exp(α * θ))^(-1)

    return f
end

In [ ]:
# Plot the f-I curve for the E/target population
gr(legendfontsize=10, labelfontsize=15, guidefontsize=15)

# Set parameters
pars = default_pars_generic_target()  # get default parameters
x = -4:0.1:10                         # set the range of input

# Compute transfer function
f = F.(x, pars.a_E, pars.theta_E)

# Visualize
plot_fI(x, f, 2, 0.2, :none, :black)
plot!(ylim=[-0.05, 1], xlabel=L"x", ylabel=L"\Phi_2(x)")

#### 1.1.2.1 Understanding the transfer function's behaviour: Effect of the parameter values $\alpha$ and $\theta$
#### 1.1.2.1.1 Effect of $\alpha$

In [ ]:
# Set parameters
pars = default_pars_generic_target()  # get default parameters
x = -4:0.1:10                         # set the range of input

αvec = 0.5:0.5:3

######################################################################
plt = plot()
for i in 1:length(αvec)
    α = αvec[i]
    # Compute transfer function
    f = F.(x, α, pars.theta_E)
    plot!(x, f, linewidth=2, c=colors[i],
            label=latexstring("$(αvec[i])"))
end
plot!(xlabel=L"x", ylabel=L"\Phi_2(x)" ,
     legendtitle=L"\alpha", guidefontsize=13, thickness_scaling=1.2,
     legend=:topleft)


#### 1.1.2.1.2 Effect of $\theta$

In [ ]:
# Set parameters
pars = default_pars_generic_target()  # get default parameters
x = -4:0.1:10                         # set the range of input

θvec = 0.5:0.5:3

######################################################################
plt = plot()
for i in 1:length(θvec)
    θ = θvec[i]
    # Compute transfer function
    f = F.(x, pars.a_E, θ)
    plot!(x, f, linewidth=2, c=colors[i],
            label=latexstring("$(θvec[i])"))
end
plot!(xlabel=L"x", ylabel=L"\Phi_2(x)",
     legendtitle=L"\theta", guidefontsize=13, thickness_scaling=1.2,
     legend=:topleft)


### 1.2 Inhibitory population

#### 1.2.1 Method 1
In this case, the transfer function for the inhibitory population was assumed to be linear as the average activity of all excitatory populations. In addition, it was also assumed that the inhibitory time constant is much faster than the excitatory time constants. As a consequence, the average firing rate of the global inhibitory population can be set to its steady state value, that is the average activity of all excitatory populations: 

$$I_{inh} = \frac{1}{p}\sum_{j=1}^p r_j$$ 

where $p$ is the number of excitatory populations. In the following, $p = 1$. Hence, according to this setup, the equations from B&L paper directly amount to study the dynamics of a 1D system in which the only variable is the firing rate of the target population of excitatory neurons.

#### 1.2.2 Method 2
Since the 1D model requires the transfer function for an excitatory population, it is useless to consider another transfer function for the inhibitory population.

### 1.3 Spontaneous activity
#### 1.3.1 Method 1
In their paper, Brunel and Lavigne apply a bias current $I_{ext}$ in order to obtain a spontaneous activity of $r_{spont} = 5$ Hz for each population. Moreover, this $r_{spont} = 5$ Hz resting state should then be stable in the sense that small perturbations applied to that state do not get amplified. This stable $r_{spont} = 5$ Hz equilibrium implies that

$$
\begin{align}
\dot r_T = 0 \quad &\leftrightarrow r_T = \Phi_1((J_1-J_I)\cdot r_T + I_{ext}) \leftrightarrow r_{spont} = \Phi_1(r_{spont} \cdot w + I_{ext})\\
\frac{d \dot r_T}{dr_T} \Biggr|_{r_T = r_{spont}} < 0 \quad &\leftrightarrow \left (-1 + w \Phi_1'(w \cdot r_{spont} + I_{ext}) \right) < 0 
\end{align}
$$
with $\Phi_1'(x)$ the first derivative of transfer function $\Phi_1(x)$, whose expression is given by

$$ \Phi_1'(x) = \tau_m\sqrt{\pi} \cdot (\Phi_1(x))^2 \cdot  \left [ \int_{-\infty}^{+\infty} z^2 \exp \left ( -x z^2- \frac{\sigma^4z^6}{48}\right)\ dz \right] $$

The conditions on $w$ and $I_{ext}$ are then
$$ I_{ext} = \Phi_1^{-1}(r_{spont})  - w \cdot r_{spont}$$

$$ w < \frac{1}{\Phi_1'(\Phi_1^{-1}(r_{spont}))} $$

with $\Phi_1^{-1}(x)$ the inverse transfer function of $\Phi_1(x)$. The inverse function is guaranteed to exist because $\Phi_1(x)$ is a monotonically increasing and continuous function.


In [ ]:
function find_Ibias!(fun, x)
    #= ===============================================================
        Find the I_bias to apply in order to have a spontaneous
        activity of r_spont with the recurrent connection w
    
    Args:
    fun    : (multivariate) function to evaluate
    x      : variable vector (here I_ext alone)
    w      : recurrent connection parameter. DEFINED BEFORE CALL !
    r_spont: spontaneous activity value [Hz]. DEFINED BEFORE CALL !
    =============================================================== =#
    fun[1] = Phi(r_spont*w + x[1]) - r_spont
end

In [ ]:
function dPhi(x, σ=0.5, τ_memb=10)
    #= ===============================================================
        Compute the first derivative of Phi(I) with respect to I
    
    Args:
    x        : Input current (float)
    σ        : Standard deviation of the fluctuation
    τ_memb   : Time constant of the cell membrane
    
    Returns:
    dphi     : The Phi' evaluated at x (float)
    =============================================================== =#
    phi = Phi(x)
    if x >= -16.0
        f(z) = z^2 * exp(-x*z^2 - σ^4*z^6/48)
        integral, error = quadgk(f, -Inf,Inf, rtol=1e-8)
        # Scaling factor 1e-3 to get sec
        dphi = sqrt(pi) * τ_memb * 1e-3 * integral
        dphi = dphi * (phi^2)
    else
        dphi = 0.0    
    end
    
    return dphi
end

In [ ]:
# Plot of the dPhi-x curve.
gr(legendfontsize=10, labelfontsize=13, guidefontsize=13, grid=false)

x = -2.0:0.01:10.
dphi = dPhi.(x)

plot(x, dphi, lw=2, c=:black, label=:none, thickness_scaling=1.2,
     xlabel=L"x",
     ylabel=L"\Phi_1'(x)")

In [ ]:
#= Determine wmax to have a spontaneous activity of r_spont Hz
   Once wmax is found, show the transition in the stability
   although using the same formula to find I_ext for r_spont Hz =#

pars = default_pars_BL_target()
@unpack r_spont, tau_E = pars
w = 0.0
I_spont = nlsolve(find_Ibias!, [-0.0]).zero[1]

wmax = 1/(dPhi(I_spont))
println("w (or J1-JI) should be less than ", wmax,
        " in order to have a stable resting state at ", r_spont,
        " Hz by choosing I_ext = I_(w=0) - w*r_spont \n")

# Compute the "rate of decay/growth" [/msec]
drdot(w, I) = (-1.0 + w * dPhi(r_spont*w + I))/tau_E 

w = wmax - 1e-3
I_spont = nlsolve(find_Ibias!, [-0.0]).zero[1]
phi_spont = Phi(w*r_spont + I_spont)

println("w = ",w)
println("I_spont = ",I_spont)
println("Phi(w*r_spont + I_spont) = ", phi_spont)
println("What about the stability for this FP ?")
println("drdot/dr = ", drdot(w, I_spont))
println("\n")

#####################################
w = wmax
I_spont = nlsolve(find_Ibias!, [-0.0]).zero[1]
phi_spont = Phi(w*r_spont + I_spont)
println("w = ",w)
println("I_spont = ",I_spont)
println("Phi(w*r_spont + I_spont) = ", phi_spont)
println("What about the stability for this FP ?")
println("drdot/dr = ", drdot(w, I_spont))
println("\n")

#####################################
w = wmax + 1e-3
I_spont = nlsolve(find_Ibias!, [-0.0]).zero[1]
phi_spont = Phi(w*r_spont + I_spont)
println("w = ",w)
println("I_spont = ",I_spont)
println("Phi(w*r_spont + I_spont) = ", phi_spont)
println("What about the stability for this FP ?")
println("drdot/dr = ", drdot(w, I_spont))

In [ ]:
# Using default values from B&L (2009) paper

pars = default_pars_BL_target()
@unpack J1, JI, r_spont, tau_E = pars
drdot(w, I) = (-1.0 + w * dPhi(r_spont*w + I))/tau_E 

w = J1-JI
I_spont = nlsolve(find_Ibias!, [-0.0]).zero[1]
phi_spont = Phi(w*r_spont + I_spont)
println("I_spont = ",I_spont)
println("Phi(w*r_spont + I_spont) = ", phi_spont)
println("What about the stability for this FP ?")
println("drdot/dr = ", drdot(w, I_spont))

In [ ]:
# Using 1-population adapted parameters: w = 0.02, r_spont = 5

pars = default_pars_BL_target()
@unpack  w, r_spont, tau_E = pars
drdot(w, I) = (-1.0 + w * dPhi(r_spont*w + I))/tau_E 

I_spont = nlsolve(find_Ibias!, [-0.0]).zero[1]
phi_spont = Phi(w*r_spont + I_spont)
println("I_spont = ",I_spont)
println("Phi(w*r_spont + I_spont) = ", phi_spont)
println("What about the stability for this FP ?")
println("drdot/dr = ", drdot(w, I_spont))

In [ ]:
#= Graphical approach for estimating wmax
   Behaviour of dṙ_T/dr_T as a function of w (= J1-JI) and 
   I_ext (r_spont is fixed and I_sel is assumed to be 0) =#

gr(legendfontsize=15, labelfontsize=16, guidefontsize=16)

wvec = range(0, 0.1, length = 20)
Ivec = range(-2.0, 20, length = 100)

pars = default_pars_BL_target()
@unpack r_spont, tau_E = pars

# Compute the "rate of decay/growth" [/msec]
drdot(w, I) = (-1.0 + w * dPhi(r_spont*w + I))/tau_E 


plt1 = surface(wvec, Ivec, drdot , c=:plasma, cbar=false,
                                              camera=(80, 20))
plot!(xlabel=L"w", ylabel=L"I_{ext}",
      zlabel=L"\frac{d \dot r_T}{dr_T}")


plt2 = surface(wvec, Ivec, drdot , c=:plasma, cbar=true,
                                              camera=(0,0))
plot!(xlabel=L"w", ylabel=L"I_{ext}", yticks=:none,
      zlabel=L"\frac{d \dot r_T}{dr_T}")

l=@layout[
    a{0.4w} b{0.6w}
]

plt = plot(plt1, plt2, layout=l, size=(900,400))

#### 1.3.2 Method 2

One could apply the same principle as above in order to set the spontaneous activity to $R$. Note however that since the generic transfer function has a lower and an upper limit saturations the spontaneous activity must be $- \frac{1}{1+\exp{(a\cdot \theta)}} \leq R \leq 1 - \frac{1}{1+\exp{(a\cdot \theta)}}$

In [ ]:
function find_generic_Ibias!(fun, x)
    #= ===============================================================
        Find the I_bias to apply in order to have a spontaneous
        activity of R with the recurent connection w
    
    Args:
    fun    : (multivariate) function to evaluate
    x      : variable vector
    w      : recurrent connection parameter
    R      : spontaneous activity value [Hz]
    =============================================================== =#
    fun[1] = F(R*w + x[1], a_E, theta_E) - R
end

In [ ]:
function dF(x, α, θ)
    #= ================================================================
        Derivative of the population activation function
    
    Args:
    x    : The population input (float scalar or vector)
    α    : The gain of the function
    θ    : The threshold of the function
    
    Returns:
    df   : 
    ================================================================ =#

    # Calculate the population activation
    dfdx = α * exp(-α * (x - θ)) * (1 + exp(-α * (x - θ)))^(-2) 
    
    return dfdx
end

In [ ]:
# Plot of the dF-I curve.
gr(legendfontsize=10, labelfontsize=12, guidefontsize=15, grid=false)

pars = default_pars_generic_target()
x = -3:0.01:10.
df = dF.(x, pars.a_E, pars.theta_E)

plot(x, df, lw=2, c=:black, label=:none, thickness_scaling=1.2,
     xlabel=L"x", ylabel=L"\Phi_2'(x)")

In [ ]:
#= Determine wmax to have a spontaneous activity of R Hz
   Once wmax found show, show the transition in the stability
   although using the same formula to find I_ext for R Hz =#

pars = default_pars_generic_target()
@unpack a_E, theta_E, tau_E = pars
R = 0.2
w = 0.0
I_spont = nlsolve(find_generic_Ibias!, [-0.0]).zero[1]

drdot_generic(w, I) = (-1 + w*dF(w*R+I, a_E, theta_E))/tau_E

wmax = 1/(dF(I_spont, a_E, theta_E))
println("w should be less than ", wmax,
        " in order to have a stable resting state at ", R,
        " Hz by choosing I_ext = I_(w=0) - w*R \n")

w = wmax - 1e-1
I_spont = nlsolve(find_generic_Ibias!, [-0.0]).zero[1]
F_spont = F(w*R + I_spont, a_E, theta_E)
println("w = ",w)
println("I_spont = ",I_spont)
println("F(w*r_spont + I_spont) = ", F_spont)
println("What about the stability for this FP ?")
println("drdot/dr = ", drdot_generic(w, I_spont))
println("\n")

#####################################
w = wmax
I_spont = nlsolve(find_generic_Ibias!, [-0.0]).zero[1]
F_spont = F(w*R + I_spont, a_E, theta_E)
println("w = ",w)
println("I_spont = ",I_spont)
println("F(w*r_spont + I_spont) = ", F_spont)
println("What about the stability for this FP ?")
println("drdot/dr = ", drdot_generic(w, I_spont))
println("\n")

#####################################
w = wmax + 1e-1
I_spont = nlsolve(find_generic_Ibias!, [-0.0]).zero[1]
F_spont = F(w*R + I_spont, a_E, theta_E)
println("w = ",w)
println("I_spont = ",I_spont)
println("F(w*r_spont + I_spont) = ", F_spont)
println("What about the stability for this FP ?")
println("drdot/dr = ", drdot_generic(w, I_spont))
println("\n")

## 2 Linear filter dynamics
### 2.1 Method 1
In the next section, simulation of the ordinary differential equation (ODE)

$$ \tau \frac{dr_T}{dt} = -r_T + \Phi_1(J_1 \cdot r_T + I_{ext} + I_{sel} - I_{inh}) $$

of the firing rate of the target population is done using the following equations for the inhibitory input current and the inhibitory firing rate.
$$ I_{inh} = J_I \cdot r_I; $$

$$ r_I = \frac{1}{p} \sum_{j=1}^p r_{j} = r_T,$$

where $J_1$ is the intrapopulation efficacy, $J_I$ is the inhibitory synaptic efficacy, $I_{ext}$ is the nonselective external input current, that is, a bias current applied in order to have a spontaneous activity of $r_{spont}$ Hz and $I_{sel}$ is the selcetive external input current, that is, the excitation current representing the presentation of the target word.

At the end, the temporal evolution of the firing rate, i.e. $r_T(t)$ is obtained.

For $1$ population only, the model to simulate amounts to 

$$ \tau \frac{dr_T}{dt} = -r_T + \Phi_1((J_1 - J_I) \cdot r_T + I_{ext} + I_{sel}) = -r_T + \Phi_1(w \cdot r_T + I) $$

In [ ]:
function simulate_BL_target(pars)
    #= ===============================================================
        Simulate the temporal dynamics of the target activity.
    
    Args:
    pars    : The parameter structure
    
    Returns:
    sol     : The solution of the ODE, i.e. t = sol.t and r(t) = sol.x
    =============================================================== =#
    # Set parameters
    @unpack tau_E, w = pars
    @unpack I_ext_E, Iapp = pars
    @unpack rE_init, T, dt = pars
    
    # Define ODE problem
    f(u,p,t) = 1/tau_E * (-u + Phi(w * u + I_ext_E + Iapp(t)))
    
    u0 = rE_init
    tspan = (0.0, T)
    prob = ODEProblem(f, u0, tspan)
    
    # Solve
    sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)
    
    return sol    
end

**Simple tests & illustration of spontaneous activity**

In [ ]:
#= Example with default parameters (i.e. w = 3.65, rE_init=3) and
   no input current (I_sel_E = 0.0). Iext is set to have an 
   equilibrium at r_spont (i.e. 0 or 5) Hz.
   The user can change T,w, r_spont, rE_init (and initial condition of
   nlsolve)
   Here, tested with w=3.65, r_spont ∈ {0,5}, rE_init ∈ {3, 5, 6} =#

pars = default_pars_BL_target()
T = 80.

# Find I_ext
w = pars.J1-pars.JI
r_spont = pars.r_spont
rE_init = pars.rE_init
I_spont = nlsolve(find_Ibias!, [-0.0]).zero[1]

#####################################################################
phi = Phi(w*r_spont + I_spont)
println("Phi(w * r_spont + I_spont) = ",phi)

# Set simulation parameters
pars = default_pars_BL_target(T=T, rE_init=rE_init, w=w, 
                              I_ext_E=I_spont, I_sel_E=0.)
sol = simulate_BL_target(pars)
phi = Phi.(pars.w * sol.u .+ pars.I_ext_E .+ pars.Iapp.(sol.t))

# Plot
plot(sol, label=L"r_{T,sim}(t)", c=colors[1], lw=2)
plot!(sol.t, phi, lw=2,c=:black, ls=:dash, label=L"\Phi_1(I_{tot})")
plot!(xlabel = L"t\ [msec]", ylabel=L"\textrm{Activity}\ r_T(t)\ [Hz]",
      legendfontsize=13, legend=:right, grid=false,
      guidefontsize=15, thickness_scaling=1.2)

In [ ]:
#= Example with 1-population adapted (i.e. w = 0.02, rE_init=3) and
   no input current (I_sel_E = 0.0). Iext is set to have an 
   equilibrium at r_spont (i.e. 5) Hz.
   The user can change T,w, r_spont, rE_init (and initial condition of
   nlsolve)
   Here, tested with w=0.02, r_spont = 5, rE_init ∈ {3, 5, 6} =#

pars = default_pars_BL_target()
T = 100.

# Find I_ext
w = pars.w
r_spont = pars.r_spont
rE_init = pars.rE_init
I_spont = nlsolve(find_Ibias!, [-0.0]).zero[1]

#####################################################################
phi = Phi(w*r_spont + I_spont)
println("Phi(w * r_spont + I_spont) = ",phi)

# Set simulation parameters
pars = default_pars_BL_target(T=T, rE_init=rE_init, w=w, 
                              I_ext_E=I_spont, I_sel_E=0.)
sol = simulate_BL_target(pars)
phi = Phi.(pars.w * sol.u .+ pars.I_ext_E .+ pars.Iapp.(sol.t))

# Plot
plot(sol, label=L"r_{T,sim}(t)", c=colors[1], lw=2)
plot!(sol.t, phi, lw=2,c=:black, ls=:dash, label=L"\Phi_1(I_{tot})")
plot!(xlabel = L"t\ [msec]", ylabel=L"\textrm{Activity}\ r_T(t)\ [Hz]",
      legendfontsize=13, legend=:right, grid=false,
      guidefontsize=15, thickness_scaling=1.2)

#### 2.1.1 Effect of rate dynamics $\tau$

The effect of rate dynamics (i.e. $\tau$) is now investigated for the case with no connectivity (i.e. $w = J_1 - J_I = 0.0$). A spontaneous activity of $5\ Hz$ is used (i.e. $I_{ext} = \Phi^{-1}(r_{spont} = 5)$) and $I_{sel} = 0.0$ for simplicity. 

Using $w=0$ reduces the equation of $\dot r_T$ to a linear low-pass filter, that is $\dot r_T$ is linear and the analytical solution is $r_T(t) = r_T(0) + (\Phi_1(I_{tot}) - r_T(0))* (1-\exp (-t/\tau))$.
Thus in the following, the numerical solution is compared with the analytical one.

In [ ]:
#= The user can change parameter values τ, T, rE_init, I_sel_E, w and 
   r_spont if desired. I_ext is then chosen to have a spontaneous 
   activty of r_spont Hz. Note however that no guarantee is given about
   the stability if w and r_spont are changed. =#

τ = 2:2:20

pars = default_pars_BL_target()

T = 100. # or pars.T
rE_init = 10. # or pars.rE_init

# Find I_ext
w = 0.0
r_spont = pars.r_spont
I_spont = nlsolve(find_Ibias!, [-0.0]).zero[1]
I_sel_E = 0 # or pars.I_sel_E
println("I_spont = ",I_spont)


######################################################################
# Plot
ln_τ = length(τ)
p = plot()
Xx = []
Yy = []

for i in 1:ln_τ
    # Set excitatory selective input and time constant tau
    pars = default_pars_BL_target(tau_E = τ[i], w = w,T = T,
                                  I_ext_E = I_spont,
                                  rE_init = rE_init, I_sel_E=I_sel_E)

    @unpack tau_E, I_ext_E, rE_init, Iapp, w = pars

    # Simulation
    sol = simulate_BL_target(pars)

    # Analytical solution; ONLY valid if J1 = JI !!!
    # i.e. Phi1 depends only on I then
    r_ana = rE_init .+ (Phi.(I_ext_E .+ Iapp.(sol.t)) .- rE_init) .*
                       (1. .- exp.(-sol.t ./ tau_E))
    
    push!(Xx, sol.t)
    push!(Yy, sol.u)

    if i < ln_τ
        p! = plot!(sol.t, r_ana, c=:gray, lw=2, ls=:dashdot,alpha=0.5,
                   label=:none, zorders=:front)
    else
        p! = plot!(sol.t, r_ana, c=:gray, lw=2, ls=:dashdot, alpha=0.5,
                   label=L"r_{T,ana}", zorders=:front)
        p! = plot!(sol.t, Phi.(w*sol.u .+ I_ext_E .+ Iapp.(sol.t)),
                   ls=:dash, lw=2, c=:black, 
                   label=L"\Phi_1(w*r_T + I_{ext}+I_{sel})")
    end
    phi = Phi(w*sol.u[length(sol.t)] + I_spont + I_sel_E)
    println("Phi(w * r_steady + I_spont + I_sel) = ",phi)
end
p! = plot!(Xx,Yy,color=cgrad(:plasma),line_z = τ',label=:none,lw=2,
           zorders=:back,cbartitle = L"\tau\ [msec]")#, cbar=false)

p! = plot!(xlabel=L"t\ [msec]",
           ylabel=L"\mathrm{Activity} \ r_T(t)\ [Hz]",
           legendfontsize=10, legend=:topright, guidefontsize=13,
           colorbar_titlefontsize=12, grid=false,
           thickness_scaling=1.2)


#### 2.1.2 Effect of synaptic current $I$

The effect of the total synaptic current (i.e. $I = I_{ext} + I_{sel}$) is now investigated. The case with no connectivity (i.e. $w = J_1 - J_I = 0.0$) is still considered to compare with the analytical solution. The individual effects of $I_{ext}$ or $I_{sel} can be obtained in a similar way by fixing one of the two to some value. The results will be the same. Here, it is therefore the effect of the synaptic current as a whole that is investigated.

In [ ]:
I = [-1.; -0.5; -0.18; 0.0; 0.5; 1.:1.:4]

pars = default_pars_BL_target()

T = 80. # or pars.T
rE_init = 10. # or pars.rE_init
w = 0.0

#####################################################################
# Plot
ln_I = length(I)
p = plot()
Xx = []
Yy = []

for i in 1:ln_I
    # Set excitatory selective input and time constant tau
    pars = default_pars_BL_target(w = w,T = T, I_ext_E = 0,
                                  rE_init = rE_init, I_sel_E=I[i])

    @unpack tau_E, I_ext_E, rE_init, Iapp, w = pars

    # Simulation
    sol = simulate_BL_target(pars)

    # Analytical solution; ONLY valid if J1 = JI !!!
    # i.e. Phi1 depends only on I then
    r_ana = rE_init .+ (Phi.(I_ext_E .+ Iapp.(sol.t)) .- rE_init) .*
                       (1. .- exp.(-sol.t ./ tau_E))
    
    push!(Xx, sol.t)
    push!(Yy, sol.u)

    if i < ln_I
        p! = plot!(sol.t, r_ana, c=:gray, lw=2, ls=:dashdot,alpha=0.5,
                   label=:none, zorders=:front)
        p! = plot!(sol.t, Phi.(w*sol.u .+ I_ext_E .+ Iapp.(sol.t)),
                   ls=:dash, lw=2, c=:black, label=:none,alpha=0.7,
                   zorders=:back)
    else
        p! = plot!(sol.t, r_ana, c=:gray, lw=2, ls=:dashdot, alpha=0.5,
                   label=L"r_{T,ana}", zorders=:front)
        p! = plot!(sol.t,
                   Phi.(pars.w*sol.u .+ I_ext_E .+ Iapp.(sol.t)),
                   ls=:dash, lw=2, c=:black, zorders=:back,alpha=0.7,
                   label=L"\Phi_1(w*r_T + I)")
    end
    phi = Phi(w*sol.u[length(sol.t)] .+ I[i])
    println("Phi_1(w * r_steady + I) = ",phi)
end
p! = plot!(Xx,Yy,color=cgrad(:plasma),line_z = I',label=:none,lw=2,
           zorders=:back,cbartitle = L"I")#, cbar=false)

p! = plot!(xlabel=L"t\ [msec]",
           ylabel=L"\mathrm{Activity} \ r_T(t)\ [Hz]",
           legendfontsize=8, legend=:outerbottom, guidefontsize=10,
           colorbar_titlefontsize=12, grid=false,
           thickness_scaling=1.2, size=(400,400))


### 2.2 Method 2
For the analogous standard method, a similar ODE is simulated showing the behaviour of a single population of excitatory neurons. This single population can be interpreted as a unique target word in working memory. (Note that the simplification assumption for $r_I$ is still used in this more standard method.)

$$\frac{dr_T}{dt} = \frac{1}{\tau}(-r_T + \Phi_2(w \cdot r_T + I)) $$

In [ ]:
function simulate_generic_target(pars)
    #= ===============================================================
        Simulate the generic equation of one single excitatory
        population of neurons
    
    Args:
    Parameters of the model
    
    Returns:
    sol : Activity of the E (target) and I populations
    =============================================================== =#
    # Set parameters
    @unpack tau_E, a_E, theta_E, wEE  = pars
    @unpack Iapp = pars
    @unpack rE_init, T, dt = pars

    f(u,p,t) = 1/tau_E * (-u + F(wEE * u + Iapp(t), a_E, theta_E))
    
    # Set simulation parameters and problem
    u0 = rE_init
    tspan = (0.0,T)
    prob = ODEProblem(f, u0, tspan)

    # Solve
    sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)

    return sol
end

**Simple tests and illustrations of spontaneous activity**

In [ ]:
#= Test different initial conditions
   Example with default parameters (wEE=9, a_E=1.2, theta_E=2.8) and
   no input current Iapp = 0.0
   Initial condition r_T(0) = 0.2 =#

T = 15.
I_ext_E = 0.0
pars = default_pars_generic_target(T=T, I_ext_E=I_ext_E)
sol1 = simulate_generic_target(pars)

plot(sol1.t, sol1.u, label=latexstring("r_T(0) = $(pars.rE_init)"),
     c=colors[1], lw=2)

# Initial conditions r_T(0) = 0.3
pars = default_pars_generic_target(T=T, I_ext_E=I_ext_E, rE_init=.3)
sol2 = simulate_generic_target(pars)
plot!(sol2.t, sol2.u, label=latexstring("r_T(0) = $(pars.rE_init)"),
      c=colors[2], lw=2)

plot!(xlabel = L"t\ [msec]", ylabel=L"\textrm{Activity}\  r_T(t)\ [Hz]",
      legendfontsize=11, legend=:right, guidefontsize=13,
      thickness_scaling=1.2, ylim=(-0.1, 1.0))

In [ ]:
#= Examples with no input current Iapp = 0.0. 
   The user can change R, w, T, I_ext_E (i.e. Iapp) and rE_init.
   I_spont is set to have a spontaneous activity of R but no guarantee
   is given about its stability. =#

# Find Ibias
pars = default_pars_generic_target()
@unpack a_E, theta_E = pars
R = 0.2
w = 2.0 #pars.wEE
I_spont = nlsolve(find_generic_Ibias!, [-0.0]).zero[1]
ff = F(w*R + I_spont, a_E, theta_E)
println("I_spont = ",I_spont)
println("F(w * R + I_spont) = ",ff)

T = 15.
I_ext_E = 0.0
rE_init1 = 0.7
rE_init2 = 0.1
rE_init3 = R

#####################################################################
# Simulate
# Initial condition rE = rE_init1
pars = default_pars_generic_target(T=T, I_ext_E=I_ext_E + I_spont,
                                   wEE=w, rE_init=rE_init1)
sol1 = simulate_generic_target(pars)

plot(sol1.t, sol1.u, label=latexstring("r_T(0)= $(rE_init1)"),
     c=colors[1], lw=2)

# Initial conditions rE = rE_init2
pars = default_pars_generic_target(T=T, I_ext_E=I_ext_E+ I_spont,
                                   wEE=w, rE_init=rE_init2)
sol2 = simulate_generic_target(pars)
plot!(sol2.t, sol2.u, label=latexstring("r_T(0)= $(rE_init2)"),
      c=colors[2], lw=2)

# Initial conditions rE = rE_init3
pars = default_pars_generic_target(T=T, I_ext_E=I_ext_E+ I_spont,
                                   wEE=w, rE_init=rE_init3)
sol2 = simulate_generic_target(pars)
plot!(sol2.t, sol2.u, label=latexstring("r_T(0)= $(rE_init3)"),
      c=colors[3], lw=2)


plot!(xlabel = L"t\ [msec]", ylabel=L"\textrm{Activity}\  r_T(t)\ [Hz]",
      legendfontsize=11, legend=:right, guidefontsize=13, grid=false,
      thickness_scaling=1.2)
plot!(ylim=(-0.1, 1.0))

#### 2.2.1 Effect of rate dynamics $\tau$

Similarly to Method 1, the effect of $\tau$ can also be investigated with Method 2. The same results can thus be concluded.

In [ ]:
#= The user can change parameter values τ, T, rE_init, Iapp, w and 
   R if desired. I_ext is then chosen to have a spontaneous 
   activity of R Hz. Note however that no guarantee is given about
   the stability if w and R are changed. =#

pars = default_pars_generic_target()
τ = [1.:1.:8.; 10]

I_ext_E = 0. #pars.I_ext_E
T = 20.
r_init = 0.5 #pars.rE_init

# Find Ibias
@unpack a_E, theta_E = pars
R = 0.0
w = 0.0 #pars.wEE
I_spont = nlsolve(find_generic_Ibias!, [-0.0]).zero[1]
println("I_spont = ",I_spont)
@printf("Minimum steady state for r_E = 1/(1 + exp(a*theta)) = %.3f\n", 
        -1. / (1. + exp(pars.a_E * pars.theta_E)))

######################################################################
# Plot
ln_τ = length(τ)
p = plot()
Xx = []
Yy = []

for i in 1:ln_τ 
    # Set excitatory selective input and time constant tau
    pars = default_pars_generic_target(I_ext_E=I_ext_E+I_spont,
                                       tau_E=τ[i], wEE=w, T=T,
                                       rE_init=r_init)

    @unpack tau_E, a_E, theta_E, rE_init, Iapp, wEE = pars

    # Simulation
    sol = simulate_generic_target(pars)

    # Analytical solution; ONLY valid if w = 0 !!!
    # i.e. F depends only on I then
    r_ana = rE_init .+ (F.(Iapp.(sol.t), a_E, theta_E) .- rE_init) .*
            (1. .- exp.(-sol.t ./ tau_E))
    
    push!(Xx, sol.t)
    push!(Yy, sol.u)

    if i < ln_τ
        p! = plot!(sol.t, r_ana, c=:gray, lw=2, ls=:dashdot,alpha=0.5,
                   label=:none, zorders=:front)
    else
        p! = plot!(sol.t, r_ana, c=:gray, lw=2, ls=:dashdot, alpha=0.5,
                   label=L"r_{T,ana}", zorders=:front)
        p! = plot!(sol.t, F.(wEE*sol.u .+  Iapp.(sol.t), a_E, theta_E),
                   ls=:dash, lw=2, c=:black, label=L"\Phi_2(w*r_T + I)")
    end
    ff = F(wEE*sol.u[end] + Iapp.(sol.t[end]), a_E, theta_E)
    println("F(w * r_steady + I) = ",ff)
end
p! = plot!(Xx,Yy,color=cgrad(:plasma),line_z = τ',label=:none,lw=2,
           zorders=:back,cbartitle = L"\tau\ [msec]")

p! = plot!(xlabel=L"t\ [msec]",
           ylabel=L"\mathrm{Activity} \ r_T(t)\ [Hz]",
           legendfontsize=11, legend=:topright, guidefontsize=13,
           colorbar_titlefontsize=13, grid=false,
           thickness_scaling=1.2)


#### 2.2.2 Effect of synaptic current $I$

Similarly to Method 1, the effect of $I$ can also be investigated with Method 2. Again, the case with null recurrent connectivity is considered and the effect of the synaptic current as a whole that is investigated. The individual effects of $I_{bias}$ or $I_{app}$ can be obtained in a similar way by fixing one of the two to some value. The results will be the same.

In [ ]:
#= The user can change T, R, wEE and I_ext_E if desired. I_spont is 
   then chosen to have R spontaneous activity but no guarantee is given
   about its stability =#

pars = default_pars_generic_target()
I_ext_E = -1.:1:7.

T = 6.
rE_init = 0.5 # or pars.rE_init
# Find Ibias
@unpack a_E, theta_E = pars
R = 0.0
w = 0. #pars.wEE
I_spont = 0 # or nlsolve(find_generic_Ibias!, [-0.0]).zero[1]
println("I_spont = ",I_spont)
@printf("Minimum steady state for r_E = 1/(1 + exp(a*theta)) = %.3f\n", 
        -1. / (1. + exp(pars.a_E * pars.theta_E)))

######################################################################
# Plots
ln_IextE = length(I_ext_E)
Xx = []
Yy = []

p = plot()
for i in 1:ln_IextE
    # Set excitatory selective input and time constant tau
    pars = default_pars_generic_target(I_ext_E=I_ext_E[i]+I_spont,
                                       wEE=w, T=T, rE_init=rE_init)
    @unpack tau_E, a_E, theta_E, rE_init, Iapp, wEE = pars

    # Simulation
    sol = simulate_generic_target(pars)
    push!(Xx, sol.t)
    push!(Yy, sol.u)

    # Analytical solution; ONLY valid if wEE = 0 !!!
    # i.e. F depends only on I_ext (+I_spont) then
    r_ana = rE_init .+ (F.(Iapp.(sol.t), a_E, theta_E) .- rE_init) .*
            (1. .- exp.(-sol.t ./ tau_E))


    if i < ln_IextE
        p! = plot!(sol.t, r_ana, c=:gray, lw=2, ls=:dashdotdot,
                   alpha=0.5, label=:none, zorders=:front)   

        p! = plot!(sol.t, F.(wEE*sol.u .+ Iapp.(sol.t), a_E, theta_E),
                   ls=:dash, lw=1, c=:black, label=:none,alpha=0.7,
                   zorders=:back)

    else
        p! = plot!(sol.t, r_ana, c=:gray, lw=2, ls=:dashdotdot,
                   alpha=0.5, zorders=:front, label=L"r_{T,ana}")

        p! = plot!(sol.t, F.(wEE*sol.u .+ Iapp.(sol.t), a_E, theta_E),
                   ls=:dash, lw=1, c=:black, alpha=0.7, 
                   label=L"\Phi_2(w*r_T + I)", zorders=:back)

    end
    ff = F(wEE*sol.u[end] + Iapp.(sol.t[end]), a_E, theta_E)
    println("F(w * r_steady + I) = ",ff)
end
p! = plot!(Xx,Yy,color=cgrad(:plasma),line_z = I_ext_E',label=:none,
           lw=2, zorders=:back,cbartitle = L"I")

p! = plot!(xlabel=L"t\ [msec]",
           ylabel=L"\mathrm{Activity} \ r_T(t)\ [Hz]",
           legendfontsize=8, legend=:outerbottom, guidefontsize=10,
           colorbar_titlefontsize=13,grid=false,
           thickness_scaling=1.2, ylims=[-0.05, 1.], size=(400,400))

## 3 Phase Portrait & Bifurcation Analyses

Unfortunately, the simple linear regime behaviour applies only with a strongly restricted set of values (a single one actually) for the recurrent connectivity $w$. However, the general case makes no assumption on the value of $w$. As a consequence, one should study the dynamics of the *nonlinear* model 
$$ \tau \frac{d r_T}{dt} = -r_T + \Phi (w\cdot r_T + I) $$
as a function of $I$ **and** $w$.
The nonlinear feature of the model does not make it possible to solve the model analytically. A phase portrait and bifurcation analysis is thus considered in order to have an idea of the qualitative behaviour of the solution.

### Geometric perspective

The cleaner approach to explore the dynamics of the model is to adopt a geometric perspective, that is, one should plot the curves $y=r_T$ and $y = \Phi (w\cdot r_T + I)$ in the same graph and study the effect of parameters $w$ and $I$. The intersection(s) of these two curves correspond(s) then to the fixed point(s) (or *attractor(s)*) of the model. When the line $y=r_T$ is above (below) the curve $y = \Phi (w\cdot r_T + I)$, it implies that $\dot{r}_T$ is negative (positive) and the firing rate will thus decrease (increase).

### 3.1 Effect of $I$

The effect of the external input current $I = I_{bias} + I_{app}$ in the nonlinear model is first investigated. This effect corresponds to a time shifting of $I$ units of the curve $y = \Phi (w\cdot r_T + I)$. This shift will be to the right (left) if $I$ is negative (positive). This effect is observed for both methods and a fixed value of $w$.

#### 3.1.1 Method 1

In [ ]:
function SN_bif_fixed_I_BL!(fct, x)
    # x[1] = r_eq, x[2] = w
    fct[1] = Phi(x[2]*x[1] + I) - x[1]
    fct[2] = x[2]*dPhi(x[2]*x[1] + I) - 1
end


function SN_bif_fixed_w_BL!(fct, x)
    # x[1] = r_eq, x[2] = I
    fct[1] = Phi(w*x[1] + x[2]) - x[1]
    fct[2] = w*dPhi(w*x[1] + x[2]) - 1
end

In [ ]:
#= Effect of I for different fixed values of w
   The user can change Ivec, w and initial conditions for nlsolve =#

gr(legendfontsize=10, labelfontsize=13, grid=false)

r = range(-100, 4000, length=4101)

Ivec= [-4000.:500.:-500;-50;0:100:500]


plt_geom1 = plot(r, r, c=:black, lw=2, label=:none,zorders=:front)

w=3.65
Xx = []
Yy = []

tmp = nlsolve(SN_bif_fixed_w_BL!, [2000., -3500.])
I_SN2 = tmp.zero[2]

tmp = nlsolve(SN_bif_fixed_w_BL!, [0., 0.])
I_SN1 = tmp.zero[2]

y1 = Phi.(w*r .+ I_SN1)
y2 = Phi.(w*r .+ I_SN2)
plot!(r,y1, fillrange = y2, fillalpha = 0.35, c = :lightgray,
      label = :none, zorders=:back)
plot!(r,y1, c=:lightseagreen, lw=2, ls=:dashdotdot,zorders=:front,
      label=L"y(r_T) = \Phi_1(w*r_T+I_{SN,1})")
plot!(r,y2, c=:skyblue, lw=2, ls=:dashdotdot,
      label=L"y(r_T) = \Phi_1(w*r_T+I_{SN,2})")

for Ii in 1:length(Ivec)
    I = Ivec[Ii]
    y = Phi.(w*r .+ I)
    push!(Xx, r)
    push!(Yy, y)
end 

plot!(title=string(latexstring("w=$(w)"),"\n",
      latexstring("I_{SN, 1}=$(Float16(I_SN1)),I_{SN, 2}=$(Float16(I_SN2))")),
      legend=false)
plot!(Xx,Yy,color=cgrad(:plasma),line_z = Ivec',label=:none,
      lw=2,cbartitle = L"I", colorbar_titlefontsize=13,zorders=:back)

#########################################
plt_geom2 = plot(r, r, c=:black, lw=2, label=:none,zorders=:front)

w=0.02
Xx = []
Yy = []

for Ii in 1:length(Ivec)
    I = Ivec[Ii]
    y = Phi.(w*r .+ I)
    push!(Xx, r)
    push!(Yy, y)
end 

plot!(title=latexstring("w=$(w)"),
      legend=false)
plot!(Xx,Yy,color=cgrad(:plasma),line_z = Ivec',label=:none,
      lw=2,cbartitle = L"I", colorbar_titlefontsize=13,zorders=:back)


#########################################
plt_geom3 = plot(r, r, c=:black, lw=2, label=:none,zorders=:front)

w=0.5
Xx = []
Yy = []

tmp = nlsolve(SN_bif_fixed_w_BL!, [200., -80.])
I_SN2 = tmp.zero[2]

tmp = nlsolve(SN_bif_fixed_w_BL!, [0., 0.])
I_SN1 = tmp.zero[2]

y1 = Phi.(w*r .+ I_SN1)
y2 = Phi.(w*r .+ I_SN2)
plot!(r,y1, fillrange = y2, fillalpha = 0.35, c = :lightgray,
      label = :none, zorders=:back)
plot!(r,y1, c=:lightseagreen, lw=2, ls=:dashdotdot,zorders=:front,
      label=L"y(r_T) = \Phi_1(w*r_T+I_{SN,1})")
plot!(r,y2, c=:skyblue, lw=2, ls=:dashdotdot,
      label=L"y(r_T) = \Phi_1(w*r_T+I_{SN,2})")

for Ii in 1:length(Ivec)
    I = Ivec[Ii]
    y = Phi.(w*r .+ I)
    push!(Xx, r)
    push!(Yy, y)
end 

plot!(title=string(latexstring("w=$(w)"),"\n",
      latexstring("I_{SN, 1}=$(Float16(I_SN1)),I_{SN, 2}=$(Float16(I_SN2))")),
      legend=false)
plot!(Xx,Yy,color=cgrad(:plasma),line_z = Ivec',label=:none,
      lw=2,cbartitle = L"I", colorbar_titlefontsize=13,zorders=:back)

#########################################
plt_geom4 = plot(r, r, c=:black, lw=2, label=:none,zorders=:front)

w=-0.05
Xx = []
Yy = []

for Ii in 1:length(Ivec)
    I = Ivec[Ii]
    y = Phi.(w*r .+ I)
    push!(Xx, r)
    push!(Yy, y)
end 

plot!(title=latexstring("w=$(w)"),
      legend=false)
plot!(Xx,Yy,color=cgrad(:plasma),line_z = Ivec',label=:none,
      lw=2,cbartitle = L"I", colorbar_titlefontsize=13,zorders=:back)

plot!(plt_geom3, plt_geom2, plt_geom1, plt_geom4,
      xlabel=L"r_T", ylabel=L"y(r_T)", titlefontsize=11,
      guidefontsize=15, cbar=false)

# Make one graph for all
l = @layout[grid(2,2) a{0.05w}]

plts = [plt_geom4 plt_geom3; plt_geom2 plt_geom1]
cbar_len = 91
tmp = range(Ivec[1], Ivec[end], length=cbar_len)

plot(plts...,heatmap(tmp.*ones(cbar_len,1), color=:plasma, 
     legend=:none, xticks=:none,
     yticks=(1:10:cbar_len, string.(tmp[1:10:cbar_len])),title=L"I"),
     layout=l, size=(700,600), margin=5px)


#### 3.1.2 Method 2

In [ ]:
function SN_bif_fixed_I_generic!(fct, x)
    # x[1] = r_eq, x[2] = w
    fct[1] = F(x[2]*x[1] + I, a_E, theta_E) - x[1]
    fct[2] = x[2]*dF(x[2]*x[1] + I, a_E, theta_E) - 1
end


function SN_bif_fixed_w_generic!(fct, x)
    # x[1] = r_eq, x[2] = I
    fct[1] = F(w*x[1] + x[2], a_E, theta_E) - x[1]
    fct[2] = w*dF(w*x[1] + x[2], a_E, theta_E) - 1
    
end

In [ ]:
#= Effect of I for different fixed values of w
   The user can change Ivec, w and initial conditions for nlsolve =#

gr(legendfontsize=10, labelfontsize=13, grid=false)

pars=default_pars_generic_target()
@unpack a_E, theta_E = pars

r = range(-0.2, 1.1, length=80)

Ivec=sort([-50:10:-20;-15:5:10;1;2:2:8])


plt_geom1 = plot(r, r, c=:black, lw=2, label=:none)

w=20
Xx = []
Yy = []

# Bistable region
tmp = nlsolve(SN_bif_fixed_w_generic!, [1., -15.])
I_SN2 = tmp.zero[2]
tmp = nlsolve(SN_bif_fixed_w_generic!, [0., 0.])
I_SN1 = tmp.zero[2]

y1 = F.(w*r .+ I_SN1, a_E, theta_E)
y2 = F.(w*r .+ I_SN2, a_E, theta_E)
plot!(r,y1, fillrange = y2, fillalpha = 0.35, c = :lightgray,
      label = :none)
plot!(r,y1, c=:lightseagreen, lw=2, ls=:dashdotdot,
      label=L"y(r_T) = \Phi_2(w*r_T+I_{SN,1})")
plot!(r,y2, c=:skyblue, lw=2, ls=:dashdotdot,
      label=L"y(r_T) = \Phi_2(w*r_T+I_{SN,2})")

for Ii in 1:length(Ivec)
    I = Ivec[Ii]
    y = F.(w*r .+ I, a_E, theta_E)
    push!(Xx, r)
    push!(Yy, y)
end 

plot!(title=string(latexstring("w=$(w)"),"\n",
      latexstring("I_{SN, 1}=$(Float16(I_SN1)),I_{SN, 2}=$(Float16(I_SN2))")),
      legend=false)
plot!(Xx,Yy,color=cgrad(:plasma),line_z = Ivec',label=:none,
      lw=2,cbartitle = L"I", colorbar_titlefontsize=13, cbar=false)


########################

plt_geom2 = plot(r, r, c=:black, lw=2, label=:none)

w=9
Xx = []
Yy = []

# Bistable region
tmp = nlsolve(SN_bif_fixed_w_generic!, [1., 1.])
I_SN2 = tmp.zero[2]
tmp = nlsolve(SN_bif_fixed_w_generic!, [0., 1.])
I_SN1 = tmp.zero[2]

y1 = F.(w*r .+ I_SN1, a_E, theta_E)
y2 = F.(w*r .+ I_SN2, a_E, theta_E)
plot!(r,y1, fillrange = y2, fillalpha = 0.35, c = :lightgray,
      label = :none)
plot!(r,y1, c=:lightseagreen, lw=2, ls=:dashdotdot,
      label=L"y(r_T) = \Phi_2(w*r_T+I_{SN,1})")
plot!(r,y2, c=:skyblue, lw=2, ls=:dashdotdot,
      label=L"y(r_T) = \Phi_2(w*r_T+I_{SN,2})")

for Ii in 1:length(Ivec)
    I = Ivec[Ii]
    y = F.(w*r .+ I, a_E, theta_E)
    push!(Xx, r)
    push!(Yy, y)
end 

plot!(title=string(latexstring("w=$(w)"),"\n",
      latexstring("I_{SN, 1}=$(Float16(I_SN1)),I_{SN, 2}=$(Float16(I_SN2))")),
      legend=false)
plot!(Xx,Yy,color=cgrad(:plasma),line_z = Ivec',label=:none,
      lw=2,cbartitle = L"I", colorbar_titlefontsize=13, cbar=false)

   
########################

plt_geom3 = plot(r, r, c=:black, lw=2, label=:none)

w=2.

Xx = []
Yy = []

for Ii in 1:length(Ivec)
    I = Ivec[Ii]
    y = F.(w*r .+ I, a_E, theta_E)
    push!(Xx, r)
    push!(Yy, y)
end 

annotate!(0.3,0.55, text(L"\dot r_T > 0 ", 11))
annotate!(0.7,0.45, text(L"\dot r_T < 0 ", 11))

plot!(title=latexstring("w=$(w)"), legend=false)
plot!(Xx,Yy,color=cgrad(:plasma),line_z = Ivec',label=:none,
      lw=2,cbartitle = L"I", colorbar_titlefontsize=13, cbar=false)


########################

plt_geom4 = plot(r, r, c=:black, lw=2, label=:none)

w=50
Xx = []
Yy = []

# Bistable region
tmp = nlsolve(SN_bif_fixed_w_generic!, [1., -40.])
I_SN2 = tmp.zero[2]
tmp = nlsolve(SN_bif_fixed_w_generic!, [0., 1.])
I_SN1 = tmp.zero[2]

y1 = F.(w*r .+ I_SN1, a_E, theta_E)
y2 = F.(w*r .+ I_SN2, a_E, theta_E)
plot!(r,y1, fillrange = y2, fillalpha = 0.35, c = :lightgray,
      label = :none)
plot!(r,y1, c=:lightseagreen, lw=2, ls=:dashdotdot,
      label=L"y(r_T) = \Phi_2(w*r_T+I_{SN,1})")
plot!(r,y2, c=:skyblue, lw=2, ls=:dashdotdot,
      label=L"y(r_T) = \Phi_2(w*r_T+I_{SN,2})")

for Ii in 1:length(Ivec)
    I = Ivec[Ii]
    y = F.(w*r .+ I, a_E, theta_E)
    push!(Xx, r)
    push!(Yy, y)
end 

plot!(title=string(latexstring("w=$(w)"),"\n",
      latexstring("I_{SN, 1}=$(Float16(I_SN1)),I_{SN, 2}=$(Float16(I_SN2))")),
      legend=false)
plot!(Xx,Yy,color=cgrad(:plasma),line_z = Ivec',label=:none,
      lw=2,cbartitle = L"I", colorbar_titlefontsize=13, cbar=false)


plot!(plt_geom3, plt_geom2, plt_geom1, plt_geom4,
      xlabel=L"r_T", ylabel=L"y(r_T)", titlefontsize=11,
      guidefontsize=15, cbar=false)

# Make one graph for all
l = @layout[grid(2,2) a{0.05w}]

plts = [plt_geom3 plt_geom1; plt_geom2 plt_geom4]
tmp = Ivec[1]:Ivec[end]
cbar_len = length(tmp)
tmp = range(Ivec[1], Ivec[end], length=cbar_len)

plot(plts...,heatmap(tmp.*ones(cbar_len,1), color=:plasma, 
     legend=:none, xticks=:none,
     yticks=(1:10:cbar_len, string.(tmp[1:10:cbar_len])),title=L"I"),
     layout=l, size=(700,700), margin=5px)


### 3.2 Effect of $w$

Similarly to the effect of $I$, the effect of the recurrent connectivity strength can be investigated. This effect corresponds to a time scaling of the curve $y = \Phi (w\cdot r_T + I)$. The curve will be contracted if $w>1$ and expanded if $w<1$. Moreover, if $w<0$, then a time folding occurs in addition to the time scaling, that is, the curve $y = \Phi (w\cdot r_T + I)$ undergoes an orthogonal symmetry with respect to $y$-axis. This effect is observed for both methods and a fixed value of $I$.

#### 3.2.1 Method 1

In [ ]:
#= Effect of w for different fixed values of I
   The user can change wvec, I and initial conditions for nlsolve =#

gr(legendfontsize=10, labelfontsize=13, grid=false)

pars=default_pars_generic_target()
@unpack a_E, theta_E = pars

r = range(-100, 4000, length=4101)

wvec = sort([-1:0.5:0; 0.02;0.04;0.5:0.5:4;6;8])
wend=100


plt_geom1 = plot(r, r, c=:black, lw=2, zorders=:front)

I=0.0
Xx = []
Yy = []

for wi in 1:length(wvec)
    w = wvec[wi]
    y = Phi.(w*r .+ I)
    push!(Xx, r)
    push!(Yy, y)
end 

plot!(title=string(latexstring("I=$(I)")))
plot!(Xx,Yy,color=cgrad(:plasma),line_z = wvec',label=:none,
      lw=2,cbartitle = L"w", colorbar_titlefontsize=13)

#######################################
plt_geom2 = plot(r, r, c=:black, lw=2, zorders=:front)

I=-2000
Xx = []
Yy = []

# Bistable regions
tmp = nlsolve(SN_bif_fixed_I_BL!, [1200, 2.5])
w_SN = tmp.zero[2]

y1 = Phi.(w_SN*r .+ I)
yend= Phi.(wend*r .+I)
plot!(r,y1, fillrange = yend,
      fillalpha = 0.35, c = :lightgray, label = :none)
plot!(r,y1, c=:lightseagreen, lw=2, ls=:dashdotdot,
      label=L"y(r_T) = \Phi_1(w_{SN}*r_T+I)")


for wi in 1:length(wvec)
    w = wvec[wi]
    y = Phi.(w*r .+ I)
    push!(Xx, r)
    push!(Yy, y)
end 

plot!(title=string(latexstring("I=$(I)"),"\n",
      latexstring("w_{SN}=$(Float16(w_SN))")),legend=false)

plot!(Xx,Yy,color=cgrad(:plasma),line_z = wvec',label=:none,
      lw=2,cbartitle = L"w", colorbar_titlefontsize=13)

########################################
plt_geom3 = plot(r, r, c=:black, lw=2, zorders=:front)

I=-4000.0
Xx = []
Yy = []

# Bistable regions
tmp = nlsolve(SN_bif_fixed_I_BL!, [2000., 4.])
w_SN = tmp.zero[2]

y1 = Phi.(w_SN*r .+ I)
yend= Phi.(wend*r .+I)
plot!(r,y1, fillrange = yend,
      fillalpha = 0.35, c = :lightgray, label = :none)
plot!(r,y1, c=:lightseagreen, lw=2, ls=:dashdotdot,
      label=L"y(r_T) = \Phi_1(w_{SN}*r_T+I)")

for wi in 1:length(wvec)
    w = wvec[wi]
    y = Phi.(w*r .+ I)
    push!(Xx, r)
    push!(Yy, y)
end 

plot!(title=string(latexstring("I=$(I)"),"\n",
      latexstring("w_{SN}=$(Float16(w_SN))")),legend=false)

plot!(Xx,Yy,color=cgrad(:plasma),line_z = wvec',label=:none,
      lw=2,cbartitle = L"w", colorbar_titlefontsize=13)

########################################
plt_geom4 = plot(r, r, c=:black, lw=2, zorders=:front)

I=100.0
Xx = []
Yy = []

for wi in 1:length(wvec)
    w = wvec[wi]
    y = Phi.(w*r .+ I)
    push!(Xx, r)
    push!(Yy, y)
end 

plot!(title=string(latexstring("I=$(I)")),legend=false)

plot!(Xx,Yy,color=cgrad(:plasma),line_z = wvec',label=:none,
      lw=2,cbartitle = L"w", colorbar_titlefontsize=13)



plot!(plt_geom3, plt_geom2, plt_geom1, plt_geom4,
      xlabel=L"r_T", ylabel=L"y(r_T)", titlefontsize=12,
      guidefontsize=15, cbar=false, legend=false,
      ylims=(-100,r[end]))

# Make one graph for all
l = @layout[grid(2,2) a{0.05w}]

plts = [plt_geom3 plt_geom1;plt_geom2 plt_geom4]
tmp = wvec[1]:wvec[end]
cbar_len = length(tmp)
tmp = range(wvec[1], wvec[end], length=cbar_len)

plot(plts...,heatmap(tmp.*ones(cbar_len,1), color=cgrad(:plasma), 
     legend=:none, xticks=:none, titlefontsize=18,
     yticks=(1:1:cbar_len, string.(tmp[1:1:cbar_len])),title=L"w"),
     layout=l, size=(700,600), margin=5px)


#### 3.2.2 Method 2

In [ ]:
#= Effect of w for different fixed values of I
   The user can change wvec, I and initial conditions for nlsolve =#

gr(legendfontsize=10, labelfontsize=13, grid=false)

pars=default_pars_generic_target()
@unpack a_E, theta_E = pars

r = range(-0.2, 1.1, length=80)

wvec = sort([-4:3.:0;1:7;10:12; 20])
wend=300


plt_geom1 = plot(r, r, c=:black, lw=2, zorders=:front)

I=0.5
Xx = []
Yy = []

# Bistable regions
tmp = nlsolve(SN_bif_fixed_I_generic!, [0.0, 40.])
w_SN1 = tmp.zero[2]
tmp = nlsolve(SN_bif_fixed_I_generic!, [0., 10.])
w_SN2 = tmp.zero[2]
tmp = nlsolve(SN_bif_fixed_I_generic!, [1., 4.])
w_SN3 = tmp.zero[2]

y1 = F.(w_SN1*r .+ I, a_E, theta_E)
y2 = F.(w_SN2*r .+ I, a_E, theta_E)
y3 = F.(w_SN3*r .+ I, a_E, theta_E)
yend = F.(wend*r .+ I, a_E, theta_E)
plot!(r,y2, fillrange = y3, fillalpha = 0.35, c = :lightgray,
      label = :none)
plot!(r,y1, fillrange = yend, fillalpha = 0.35, c = :lightgray,
      label = :none)
plot!(r,y1, c=:lightseagreen, lw=2, ls=:dashdotdot,
      label=L"y(r_T) = \Phi_2(w_{SN,1}*r_T+I)")
plot!(r,y2, c=:skyblue, lw=2, ls=:dashdotdot,
      label=L"y(r_T) = \Phi_2(w_{SN,2}*r_T+I)")
plot!(r, y3, c=:lightgreen, lw=2, ls=:dashdotdot,
      label=L"y(r_T) = \Phi_2(w_{SN,3}*r_T+I)")


for wi in 1:length(wvec)
    w = wvec[wi]
    y = F.(w*r .+ I, a_E, theta_E)
    push!(Xx, r)
    push!(Yy, y)
end 

plot!(title=string(latexstring("I=$(I)"),"\n",
      latexstring("w_{SN1}=$(Float16(w_SN1)), w_{SN2}=$(Float16(w_SN2))"),"\n",
      latexstring("w_{SN3}=$(Float16(w_SN3))")),
      legend=false)
plot!(Xx,Yy,color=cgrad(:plasma),line_z = wvec',label=:none,
      lw=2,cbartitle = L"w", colorbar_titlefontsize=13)


#################################
plt_geom2 = plot(r, r, c=:black, lw=2, zorders=:front)

I=0.0 # r_T=0 changes stability when w > 26
Xx = []
Yy = []

tmp = nlsolve(SN_bif_fixed_I_generic!, [1., 4.])
w_SN = tmp.zero[2]
y1 = F.(w_SN*r .+ I, a_E, theta_E)
y2 = F.(wend*r .+ I, a_E, theta_E)
plot!(r,y1, fillrange = y2, fillalpha = 0.35, c = :lightgray,
      label = :none)
plot!(r,y1, c=:lightseagreen, lw=2, ls=:dashdotdot,
      label=L"y(r_T) = \Phi_2(w_{SN}*r_T+I)")

for wi in 1:length(wvec)
    w = wvec[wi]
    y = F.(w*r .+ I, a_E, theta_E)
    push!(Xx, r)
    push!(Yy, y)
end 

plot!(title=string(latexstring("I=$(I)"),"\n",
      latexstring("w_{SN}=$(Float16(w_SN))")),
      legend=false)
plot!(Xx,Yy,color=cgrad(:plasma),line_z = wvec',label=:none,
      lw=2,cbartitle = L"w", colorbar_titlefontsize=13)


#################################
plt_geom3 = plot(r, r, c=:black, lw=2, zorders=:front)

I=1.5
Xx = []
Yy = []

tmp = nlsolve(SN_bif_fixed_I_generic!, [-0.03, 200.])
w_SN = tmp.zero[2]

y1 = F.(w_SN*r .+ I, a_E, theta_E)
y2 = F.(wend*r .+ I, a_E, theta_E)
plot!(r,y1, fillrange = y2, fillalpha = 0.35, c = :lightgray,
      label = :none)
plot!(r,y1, c=:lightseagreen, lw=2, ls=:dashdotdot,
      label=L"y(r_T) = \Phi_2(w_{SN}*r_T+I)")

for wi in 1:length(wvec)
    w = wvec[wi]
    y = F.(w*r .+ I, a_E, theta_E)
    push!(Xx, r)
    push!(Yy, y)
end 

plot!(title=string(latexstring("I=$(I)"),"\n",
      latexstring("w_{SN}=$(Float16(w_SN))")),
      legend=false)
plot!(Xx,Yy,color=cgrad(:plasma),line_z = wvec',label=:none,
      lw=2,cbartitle = L"w", colorbar_titlefontsize=13)


#################################
plt_geom4 = plot(r, r, c=:black, lw=2, zorders=:front)

I=-1.0
Xx = []
Yy = []

tmp = nlsolve(SN_bif_fixed_I_generic!, [0.75, 10.])
w_SN = tmp.zero[2]

y1 = F.(w_SN*r .+ I, a_E, theta_E)
y2 = F.(wend*r .+ I, a_E, theta_E)
plot!(r,y1, fillrange = y2, fillalpha = 0.35, c = :lightgray,
      label = :none)
plot!(r,y1, c=:lightseagreen, lw=2, ls=:dashdotdot,
      label=L"y(r_T) = \Phi_2(w_{SN}*r_T+I)")

for wi in 1:length(wvec)
    w = wvec[wi]
    y = F.(w*r .+ I, a_E, theta_E)
    push!(Xx, r)
    push!(Yy, y)
end 

plot!(title=string(latexstring("I=$(I)"),"\n",
      latexstring("w_{SN}=$(Float16(w_SN))")),
      legend=false)
plot!(Xx,Yy,color=cgrad(:plasma),line_z = wvec',label=:none,
      lw=2,cbartitle = L"w", colorbar_titlefontsize=13)

plot!(plt_geom3, plt_geom2, plt_geom1, plt_geom4,
      xlabel=L"r_T", ylabel=L"y(r_T)", titlefontsize=12,
      guidefontsize=15, cbar=false)

# Make one graph for all
l = @layout[grid(2,2) a{0.05w}]

plts = [plt_geom4 plt_geom1; plt_geom2 plt_geom3]
tmp = wvec[1]:wvec[end]
cbar_len = length(tmp)
tmp = range(wvec[1], wvec[end], length=cbar_len)

plot(plts...,heatmap(tmp.*ones(cbar_len,1), color=:plasma, 
     legend=:none, xticks=:none, titlefontsize=18,
     yticks=(5:5:cbar_len, string.(tmp[5:5:cbar_len])),title=L"w"),
     layout=l, size=(700,600), margin=5px)


### Bifurcation Diagrams

The geometric perspective is a very useful tool to understand the global behaviour of the vector field, and to determine the number of intersections (thus equilibria of the model). This perspective is also useful to investigate the individual effects of each parameter considering the others as frozen. Based on this perspective, it has been seen that the model, in both methods, changes completely its behaviour (monostable $\leftrightarrow$ bistable) for a range of parameters' values. A drawback of this geometric perspective though is that it does not give clear information about the exact value of the equilibria. This the reason why *bifurcation diagrams* are drawn. Bifurcation diagrams show the equilibria values as a function of one parameter, the others being considered as frozen again. In other words, bifurcation diagrams show the curve(s) $\dot{r_T} = 0$ as a function of a parameter. These diagrams give thus *complementary* information to the geometric perspective/phase portrait graphs.

### 3.3 $\{I,r_T^*\}-$ Bifurcation diagrams
The bifurcation diagrams are once again drawn for both methods and different fixed values of $w$.

#### 3.3.1 Method 1

In [ ]:
# {I,r_T^*}-Bifurcation diagram for w < 1/Φ'max
# Test with 1-population adapted parameter w = 0.02

pars =default_pars_BL_target()
@unpack w = pars

w = w
Ivec = range(-100, 100, length=200)
rvec = range(-50, 350, length=200)

# Simulations to determine stability of branches
T = 50
r_init = 100
Iapp1 = -50
Iapp2 = 50

######################################################################
gr(labelfontsize=13)

rnull(I,r) = -r + Phi(w*r + I)

pars1 = default_pars_BL_target(T=T, w=w, rE_init=r_init, I_ext_E=0,
                                   I_sel_E=Iapp1)
sol1 = simulate_BL_target(pars1)
I1 = pars1.I_ext_E .+ pars1.Iapp.(sol1.t)

pars2 = default_pars_BL_target(T=T, w=w, rE_init=r_init, I_ext_E=0,
                              I_sel_E=Iapp2)
sol2 = simulate_BL_target(pars2)
I2 = pars2.I_ext_E .+ pars2.Iapp.(sol2.t)

@time plt = contour(Ivec, rvec, rnull, levels=[0], cbar=false,
                    c=:black, lw=2)
plot!(I1, sol1.u, c=colors[1], lw=2, label=:none)
plot!(I2, sol2.u, c=colors[1], lw=2, label=:none)
scatter!([I1[1], I2[1]], [sol1.u[1], sol2.u[1]], c=colors[1],
         ms=4,msc=colors[1], label=:none)
annotate!(10,150,text(L"\textrm{stable}", 11, :blue, rotation=50))

plot!(xlabel=L"I", ylabel=L"r_T^*", grid=false, size=(300,200))


In [ ]:
# {I,r_T^*}-Bifurcation diagram for other w > 1/Φ'max (=0.0345)

w = 0.5
Ivec = range(-100, 10, length=100)
rvec = range(-50, 550, length=500)

# Simulations to determine stability of branches
T = 120
r_init = 100
Iapp = [-70, -50, -20, 5]
tmp1 = nlsolve(SN_bif_fixed_w_BL!, [0., 0.]) # I_SN1 (r_T* = 0.1787)
tmp2 = nlsolve(SN_bif_fixed_w_BL!, [300., -60.]) # I_SN2 (r_T* = 253.3)

######################################################################
gr(labelfontsize=13)

rnull(I,r) = -r + Phi(w*r + I)

@time plt = contour(Ivec, rvec, rnull, levels=[0], cbar=false,
                    c=:black, lw=2)
for iapp in Iapp
    pars = default_pars_BL_target(T=T, w=w, rE_init=r_init, I_ext_E=0,
                                  I_sel_E=iapp)
    sol = simulate_BL_target(pars)
    I = pars.I_ext_E .+ pars.Iapp.(sol.t)
    plot!(I, sol.u, c=colors[1], lw=2, label=:none)
    scatter!([I[1]], [sol.u[1]], c=colors[1], ms=4,msc=colors[1],
             label=:none)
end
annotate!(-85,40,text(L"\textrm{stable}", 11, :blue, rotation=0))
annotate!(-40,450,text(L"\textrm{stable}", 11, :blue, rotation=20))
annotate!(-37,130,text(L"\textrm{unstable}", 11, :red, rotation=-20))
scatter!([tmp1.zero[2]], [tmp1.zero[1]], c=:violet, ms=4, msc=:violet,
          label=:none)
scatter!([tmp2.zero[2]], [tmp2.zero[1]], c=:violet, ms=4, msc=:violet,
          label=:none)
annotate!(tmp1.zero[2]+12,tmp1.zero[1],
          text(L"I_{SN,1}", 10, :violet, rotation=0))
annotate!(tmp2.zero[2]-10,tmp2.zero[1],
          text(L"I_{SN,2}", 10, :violet, rotation=0))

plot!(xlabel=L"I", ylabel=L"r_T^*", grid=false, size=(300,200))


In [ ]:
# {I,r_T^*}-Bifurcation diagram for default values of B&L paper
# (i.e. w = 3.65)

pars =default_pars_BL_target()
@unpack J1, JI = pars

w = J1-JI
Ivec = range(-4000, 100, length=1000)
rvec = range(0, 4000, length=1000)

# Simulations to determine stability of branches
T = 120
r_init = 500
Iapp = [-3700, -2000, -1000, 10]
tmp1 = nlsolve(SN_bif_fixed_w_BL!, [0., 0.]) # I_SN1 (r_T* = 0.0207)
tmp2 = nlsolve(SN_bif_fixed_w_BL!, [1800., -3500.]) # I_SN2 (r_T* =1849.1116)

######################################################################
gr(labelfontsize=13)

rnull(I,r) = -r + Phi(w*r + I)

@time plt = contour(Ivec, rvec, rnull, levels=[0], cbar=false,
                    c=:black, lw=2)

for iapp in Iapp
    pars = default_pars_BL_target(T=T, w=w, rE_init=r_init, I_ext_E=0,
                                  I_sel_E=iapp)
    sol = simulate_BL_target(pars)
    I = pars.I_ext_E .+ pars.Iapp.(sol.t)
    plot!(I, sol.u, c=colors[1], lw=2, label=:none)
    scatter!([I[1]], [sol.u[1]], c=colors[1], ms=4,msc=colors[1],
             label=:none)
end
annotate!(-3000,300,text(L"\textrm{stable}", 11, :blue, rotation=0))
annotate!(-2000,3300,text(L"\textrm{stable}", 11, :blue, rotation=15))
annotate!(-2000,1000,text(L"\textrm{unstable}", 11, :red, rotation=-15))
scatter!([tmp1.zero[2]], [tmp1.zero[1]], c=:violet, ms=4, msc=:violet,
          label=:none)
scatter!([tmp2.zero[2]], [tmp2.zero[1]], c=:violet, ms=4, msc=:violet,
          label=:none)
annotate!(tmp1.zero[2],tmp1.zero[1]+300,
          text(L"I_{SN,1}", 10, :violet, rotation=0))
annotate!(tmp2.zero[2]+450,tmp2.zero[1]-10,
          text(L"I_{SN,2}", 10, :violet, rotation=0))

plot!(xlabel=L"I", ylabel=L"r_T^*", ylim=(-200,rvec[end]), grid=false,
      size=(300,200))

#### 3.3.2 Method 2

In [ ]:
# {I,r_T^*}-Bifurcation diagram for w < 1/Φmax' (i.e. w = 2)

pars =default_pars_generic_target()
@unpack a_E, theta_E = pars

w = 2
Ivec = range(-5, 10, length=50)
rvec = range(-0.1, 1, length=50)

# Simulations to determine stability of branches
T = 50
r_init = 0.25
Iapp = [-1,1,2.5,6]

######################################################################
gr(labelfontsize=13, size=(300,200))

rnull(I,r) = -r + F(w*r + I, a_E, theta_E)

@time plt = contour(Ivec, rvec, rnull, levels=[0], cbar=false,
                    c=:black, lw=2)

for iapp in Iapp
    pars = default_pars_generic_target(T=T, wEE=w, rE_init=r_init,
                                       I_ext_E=iapp)
    sol = simulate_generic_target(pars)
    I = pars.Iapp.(sol.t)
    plot!(I, sol.u, c=colors[1], lw=2, label=:none)
    scatter!([I[1]], [sol.u[1]], c=colors[1], ms=4,msc=colors[1],
             label=:none)
end
annotate!(-3,0.03,text(L"\textrm{stable}", 11, :blue, rotation=0))

plot!(xlabel=L"I", ylabel=L"r_T^*", ylim=(rvec[1],rvec[end]),
      grid=false)

In [ ]:
# {I,r_T^*}-Bifurcation diagram for default value (i.e. w = 9)

pars = default_pars_generic_target()
@unpack a_E, theta_E, wEE = pars

w = wEE
Ivec = range(-5, 2.5, length=50)
rvec = range(-0.1, 1, length=50)

# Simulations to determine stability of branches
T = 50
r_init = 0.25
Iapp = [-4,-1,0,1]
tmp1 = nlsolve(SN_bif_fixed_w_generic!, [-3., 0.8]) # ISN1 (r* =0.07)
tmp2 = nlsolve(SN_bif_fixed_w_generic!, [0.3, 0.3]) # ISN2 (r* =0.8631)

######################################################################
gr(labelfontsize=13, size=(300,200))

rnull(I,r) = -r + F(w*r + I, a_E, theta_E)

@time plt = contour(Ivec, rvec, rnull, levels=[0], cbar=false,
                    c=:black, lw=2)

for iapp in Iapp
    pars = default_pars_generic_target(T=T, wEE=w, rE_init=r_init,
                                       I_ext_E=iapp)
    sol = simulate_generic_target(pars)
    I = pars.Iapp.(sol.t)
    plot!(I, sol.u, c=colors[1], lw=2, label=:none)
    scatter!([I[1]], [sol.u[1]], c=colors[1], ms=4,msc=colors[1],
             label=:none)
end
annotate!(-2.5,0.03,text(L"\textrm{stable}", 11, :blue, rotation=0))
annotate!(-1,1.03,text(L"\textrm{stable}", 11, :blue, rotation=0))
annotate!(-1.3,0.55,text(L"\textrm{unstable}", 11, :red, rotation=-40))
scatter!([tmp1.zero[2]], [tmp1.zero[1]], c=:violet, ms=4, msc=:violet,
          label=:none)
scatter!([tmp2.zero[2]], [tmp2.zero[1]], c=:violet, ms=4, msc=:violet,
          label=:none)
annotate!(tmp1.zero[2]+0.8,tmp1.zero[1],
          text(L"I_{SN,1}", 10, :violet, rotation=0))
annotate!(tmp2.zero[2]-0.8,tmp2.zero[1],
          text(L"I_{SN,2}", 10, :violet, rotation=0))


plot!(xlabel=L"I", ylabel=L"r_T^*", ylim=(rvec[1],rvec[end]),
      grid=false)

In [ ]:
# {I,r}-Bifurcation diagram for w > 1/F'(F^-1(0))
pars =default_pars_generic_target()
@unpack a_E, theta_E = pars

w = 50.0
Ivec = range(-50, 5, length=80)
rvec = range(-0.1, 1, length=50)


# Simulations to determine stability of branches
T = 50
r_init = 0.25
Iapp = [-45,-20,-5,3]
tmp1 = nlsolve(SN_bif_fixed_w_generic!, [0., 0]) # I_SN1 (r_T* =-0.0166)
tmp2 = nlsolve(SN_bif_fixed_w_generic!, [1., -40.]) # I_SN2 (r_T*=0.9495)

######################################################################
gr(labelfontsize=13, size=(300,200))

rnull(I,r) = -r + F(w*r + I, a_E, theta_E)

@time plt = contour(Ivec, rvec, rnull, levels=[0], cbar=false,
                    c=:black, lw=2)

for iapp in Iapp
    pars = default_pars_generic_target(T=T, wEE=w, rE_init=r_init,
                                       I_ext_E=iapp)
    sol = simulate_generic_target(pars)
    I = pars.Iapp.(sol.t)
    plot!(I, sol.u, c=colors[1], lw=2, label=:none)
    scatter!([I[1]], [sol.u[1]], c=colors[1], ms=4,msc=colors[1],
             label=:none)
end
annotate!(-35,0.03,text(L"\textrm{stable}", 11, :blue, rotation=0))
annotate!(-20,1.03,text(L"\textrm{stable}", 11, :blue, rotation=0))
annotate!(-25,0.65,text(L"\textrm{unstable}", 11, :red, rotation=-35))
scatter!([tmp1.zero[2]], [tmp1.zero[1]], c=:violet, ms=4, msc=:violet,
          label=:none)
scatter!([tmp2.zero[2]], [tmp2.zero[1]], c=:violet, ms=4, msc=:violet,
          label=:none)
annotate!(tmp1.zero[2]+4,tmp1.zero[1]+0.07,
          text(L"I_{SN,1}", 10, :violet, rotation=0))
annotate!(tmp2.zero[2]-5,tmp2.zero[1],
          text(L"I_{SN,2}", 10, :violet, rotation=0))

plot!(xlabel=L"I", ylabel=L"r_T^*", ylim=(rvec[1],rvec[end]),
      grid=false)

### 3.4 $\{w,r_T^*\}-$ Bifurcation diagrams
Similarly to $\{I,r_T^*\}-$ bifurcation diagrams, the $\{w,r_T^*\}-$ bifurcation diagrams can also be drawn by considering the value of $I$ frozen.

The bifurcation diagrams are once again drawn for both methods and different fixed values of $w$.

#### 3.4.1 Method 1

In [ ]:
# {w,r}-Bifurcation diagram for I < I_SN2
I = -4000.

wvec = range(-10, 100, length=120)
rvec = range(0, 4000, length=2000)

# Simulations to determine stability of branches
T = 60
r_init = 500
w = [-5.,10.]
tmp = nlsolve(SN_bif_fixed_I_BL!, [2000, 3.9]) # I_SN (r_T* =2013.168)

######################################################################
gr(labelfontsize=13, size=(300,200))

rnull(ww,r) = -r + Phi(ww*r + I)

@time plt = contour(wvec, rvec, rnull, levels=[0], cbar=false,
                    c=:black, lw=2)

for wi in w
    pars = default_pars_BL_target(T=T, w=wi, rE_init=r_init,I_ext_E=0,
                                  I_sel_E=I)
    sol = simulate_BL_target(pars)
    plot!(wi*ones(length(sol.t)), sol.u, c=colors[1], lw=2,
          label=:none)
    scatter!([wi], [sol.u[1]], c=colors[1], ms=4,msc=colors[1],
             label=:none)
end
# Extra simulation with different r_init
r_init = -500
w = 10.
pars = default_pars_BL_target(T=T, w=w, rE_init=r_init,I_ext_E=0,
                              I_sel_E=I)
sol = simulate_BL_target(pars)
plot!(w*ones(length(sol.t)), sol.u, c=colors[1], lw=2, label=:none)
scatter!([w], [sol.u[1]], c=colors[1], ms=4,msc=colors[1],label=:none)

# Annotations
annotate!(50,-200,text(L"\textrm{stable}", 11, :blue, rotation=0))
annotate!(75,300,text(L"\textrm{unstable}", 11, :red, rotation=0))
plot!([25, 5],[3000,3000], arrow=(true,:closed), c=:lightgray, lw=2,
      label=:none)
annotate!(38,3100,text(L"\textrm{stable}", 11, :blue, rotation=0))
    
scatter!([tmp.zero[2]], [tmp.zero[1]], c=:violet, ms=3, msc=:violet,
          label=:none)
annotate!(-5,tmp.zero[1]+20,text(L"w_{SN}", 10, :violet, rotation=0))


plot!(xlabel=L"w", ylabel=L"r_T^*", ylim=(-1000,rvec[end]), grid=false)


In [ ]:
# {w,r}-Bifurcation diagram for I in bistable region
I = -10.

wvec = range(-2, 10, length=100)
rvec = range(0, 4000, length=5000)

# Simulations to determine stability of branches
T = 60
r_init = 500
w = [-1.,1.]
tmp = nlsolve(SN_bif_fixed_I_BL!, [100, 0.2]) # I_SN (r_T* =100.657)

######################################################################
gr(labelfontsize=13, size=(300,200))

rnull(ww,r) = -r + Phi(ww*r + I)

@time plt = contour(wvec, rvec, rnull, levels=[0], cbar=false,
                    c=:black, lw=2)

for wi in w
    pars = default_pars_BL_target(T=T, w=wi, rE_init=r_init,I_ext_E=0,
                                  I_sel_E=I)
    sol = simulate_BL_target(pars)
    plot!(wi*ones(length(sol.t)), sol.u, c=colors[1], lw=2,
          label=:none)
    scatter!([wi], [sol.u[1]], c=colors[1], ms=4,msc=colors[1],
             label=:none)
end
# Extra simulation with different r_init
r_init = -400
w = 3
pars = default_pars_BL_target(T=T, w=w, rE_init=r_init,I_ext_E=0,
                              I_sel_E=I)
sol = simulate_BL_target(pars)
plot!(w*ones(length(sol.t)), sol.u, c=colors[1], lw=2,
      label=:none)
scatter!([w], [sol.u[1]], c=colors[1], ms=4,msc=colors[1],
         label=:none)

# Annotations
annotate!(6,-100,text(L"\textrm{stable}", 11, :blue, rotation=0))
annotate!(0.5,1100,text(L"\textrm{stable}", 11, :blue, rotation=70))
annotate!(5,150,text(L"\textrm{unstable}", 11, :red, rotation=0))

scatter!([tmp.zero[2]], [tmp.zero[1]], c=:violet, ms=3, msc=:violet,
          label=:none)
annotate!(0.5,-200,text(L"w_{SN}", 11, :violet, rotation=0))


plot!(xlabel=L"w", ylabel=L"r_T^*", ylim=(-500,2000), grid=false)


In [ ]:
# {w,r}-Bifurcation diagram for I > I_SN1

I = 0.

wvec = range(-2, 4, length=50)
rvec = range(0, 4000, length=2000)

# Simulations to determine stability of branches
T = 60
r_init = 1000
w = [-1.,2.]

######################################################################
gr(labelfontsize=13, size=(300,200))

rnull(ww,r) = -r + Phi(ww*r + I)

@time plt = contour(wvec, rvec, rnull, levels=[0], cbar=false,
                    c=:black, lw=2)
for wi in w
    pars = default_pars_BL_target(T=T, w=wi, rE_init=r_init,I_ext_E=0,
                                  I_sel_E=I)
    sol = simulate_BL_target(pars)
    plot!(wi*ones(length(sol.t)), sol.u, c=colors[1], lw=2,
          label=:none)
    scatter!([wi], [sol.u[1]], c=colors[1], ms=4,msc=colors[1],
             label=:none)
end
annotate!(1,1350,text(L"\textrm{stable}", 11, :blue, rotation=45))


plot!(xlabel=L"w", ylabel=L"r_T^*", ylim=(-200,rvec[end]), grid=false)


#### 3.4.2 Method 2

In [ ]:
# {w,r}-Bifurcation diagram for I_ext = -1

pars =default_pars_generic_target()
@unpack a_E, theta_E = pars

I = -1.
wvec = range(-50, 200, length=200)
rvec = range(-0.1, 1, length=200)

# Simulations to determine stability of branches
T = 60
r_init = 0.1
w = [-30, 15,50]
tmp = nlsolve(SN_bif_fixed_I_generic!, [1., 10.]) # I_SN (r_T* =0.813)

######################################################################
gr(labelfontsize=13)

rnull(w,r) = -r + F(w*r + I, a_E, theta_E)

@time plt = contour(wvec, rvec, rnull, levels=[0], cbar=false, c=:black, lw=2)
for wi in w
    pars = default_pars_generic_target(T=T, wEE=wi, rE_init=r_init,
                                       I_ext_E=I)
    sol = simulate_generic_target(pars)
    plot!(wi*ones(length(sol.t)), sol.u, c=colors[1], lw=2,
          label=:none)
    scatter!([wi], [sol.u[1]], c=colors[1], ms=4,msc=colors[1],
             label=:none)
end
annotate!(100,-0.1,text(L"\textrm{stable}", 11, :blue, rotation=0))
annotate!(100,1.05,text(L"\textrm{stable}", 11, :blue, rotation=0))
annotate!(100,0.1,text(L"\textrm{unstable}", 11, :red, rotation=0))
        
scatter!([tmp.zero[2]], [tmp.zero[1]], c=:violet, ms=4, msc=:violet,
          label=:none)
annotate!(tmp.zero[2]-20,tmp.zero[1]-0.05,
          text(L"w_{SN}", 11, :violet, rotation=0))

plot!(xlabel=L"w", ylabel=L"r_T^*", ylim=(-0.25,rvec[end]),
      grid=false)

In [ ]:
# {w,r}-Bifurcation diagram for default I_ext = 0

pars =default_pars_generic_target()
@unpack I_ext_E, a_E, theta_E = pars

I = I_ext_E
wvec = range(-10, 50, length=1000)
rvec = range(-0.1, 1, length=1000)

# Find TC point
R=0
w=0
tmp = nlsolve(find_generic_Ibias!, [0.0]) # I_SN (r_T* =0.764)
Ispont = tmp.zero[1]
wTC = 1/dF(Ispont, a_E, theta_E)

# Simulations to determine stability of branches
T = 60
r_init = 0.15
w = [0, 10,20,40]
tmp = nlsolve(SN_bif_fixed_I_generic!, [1, 5.]) # I_SN (r_T* =0.764)

######################################################################
gr(labelfontsize=13, size=(300,200))

rnull(w,r) = -r + F(w*r + I, a_E, theta_E)

@time plt = contour(wvec, rvec, rnull, levels=[0], cbar=false,
                    c=:black, lw=2)

for wi in w
    pars = default_pars_generic_target(T=T, wEE=wi, rE_init=r_init,
                                       I_ext_E=I)
    sol = simulate_generic_target(pars)
    plot!(wi*ones(length(sol.t)), sol.u, c=colors[1], lw=2,
          label=:none)
    scatter!([wi], [sol.u[1]], c=colors[1], ms=4,msc=colors[1],
             label=:none)
end

annotate!(5,-0.05,text(L"\textrm{stable}", 11, :blue, rotation=0))
annotate!(30,1.03,text(L"\textrm{stable}", 11, :blue, rotation=0))
annotate!(40,-0.1,text(L"\textrm{stable}", 11, :blue, rotation=-3))
annotate!(40,0.08,text(L"\textrm{unstable}", 11, :red, rotation=0))
annotate!(9,0.45,text(L"\textrm{unstable}", 11, :red, rotation=-80))

scatter!([tmp.zero[2]], [tmp.zero[1]], c=:violet, ms=4, msc=:violet,
          label=:none)
annotate!(tmp.zero[2]-5,tmp.zero[1],
          text(L"w_{SN}", 11, :violet, rotation=0))

scatter!([wTC], [R], c=:violet, ms=4, msc=:violet, label=:none)
annotate!(wTC,R-0.1,
          text(L"w_{TC}", 11, :violet, rotation=0))
                    
plot!(xlabel=L"w", ylabel=L"r_T^*", ylim=(-0.25,rvec[end]), grid=false)


In [ ]:
# {w,r}-Bifurcation diagram for I_ext = 0.5

pars =default_pars_generic_target()
@unpack a_E, theta_E = pars

I = 0.5
wvec = range(-50, 150, length=100)
rvec = range(-0.1, 1, length=60)

# Simulations to determine stability of branches
T = 60
r_init = 0.6
w = [-30, 6,50,100]
tmp1 = nlsolve(SN_bif_fixed_I_generic!, [0, 100.]) # I_SN (r_T* =-0.02)
tmp2 = nlsolve(SN_bif_fixed_I_generic!, [0.05, 0.05]) # I_SN (r_T* =0.095)
tmp3 = nlsolve(SN_bif_fixed_I_generic!, [1, 0.]) # I_SN (r_T* =0.72)

######################################################################
gr(labelfontsize=13, size=(300,200))

rnull(w,r) = -r + F(w*r + I, a_E, theta_E)

@time plt = contour(wvec, rvec, rnull, levels=[0], cbar=false,
                    c=:black, lw=2)

for wi in w
    pars = default_pars_generic_target(T=T, wEE=wi, rE_init=r_init,
                                       I_ext_E=I)
    sol = simulate_generic_target(pars)
    plot!(wi*ones(length(sol.t)), sol.u, c=colors[1], lw=2,
          label=:none)
    scatter!([wi], [sol.u[1]], c=colors[1], ms=4,msc=colors[1],
             label=:none)
end
annotate!(-20,-0.05,text(L"\textrm{stable}", 11, :blue, rotation=0))
annotate!(50,1.05,text(L"\textrm{stable}", 11, :blue, rotation=0))
annotate!(110,-0.1,text(L"\textrm{stable}", 11, :blue, rotation=0))
annotate!(120,0.05,text(L"\textrm{unstable}", 11, :red, rotation=0))
annotate!(0,0.4,text(L"\textrm{unstable}", 11, :red, rotation=-87))
                
scatter!([tmp1.zero[2]], [tmp1.zero[1]], c=:violet, ms=4, msc=:violet,
          label=:none)
annotate!(tmp1.zero[2]-5,tmp1.zero[1]-0.08,
          text(L"w_{SN,1}", 11, :violet, rotation=0))
scatter!([tmp2.zero[2]], [tmp2.zero[1]], c=:violet, ms=4, msc=:violet,
          label=:none)
annotate!(tmp2.zero[2]+25,tmp2.zero[1]+0.03,
          text(L"w_{SN,2}", 11, :violet, rotation=0))
scatter!([tmp3.zero[2]], [tmp3.zero[1]], c=:violet, ms=4, msc=:violet,
          label=:none)
annotate!(tmp3.zero[2]-25,tmp3.zero[1]+0.03,
          text(L"w_{SN,3}", 11, :violet, rotation=0))

plot!(xlabel=L"w", ylabel=L"r_T^*", ylim=(-0.25,rvec[end]),
      grid=false)


### 3.5 Stability diagrams
A drawback common to both the geometric perspective and the bifurcation analysis is that one of the two parameters must be frozen in order to assess the effect of the other parameter. In addition, it has been seen that **both** parameters **influence** the number of equilibria of the model. One could therefore wonder how does the number of fixed points behave as *both* parameters vary? The answer to this question lies in the *stability diagram* that shows the number of equilibria as a function of the two independent parameters ($w$ and $I$).

In order to determine this stability diagram, one needs to compute numerically the fixed point(s) for a particular pair $(w,I)$ and repeat for a large number of different pairs.

#### 3.5.1 Method 1

In [ ]:
function fp_BL!(fct,x)
    global I, w
    fct[1] = -x[1] + Phi(w*x[1] + I)
end

function jacobian_BL!(J, x)
    global I, w
    J[1] = (-1.0 + w*dPhi(w*x[1] + I))/tau_E
    
end

In [ ]:
# Method 1 - (I,w) Stability diagram
gr(labelfontsize=13, size=(300,200))
pars = default_pars_BL_target()
@unpack tau_E = pars

x0s = range(-5, 2000, length=4000)


Ivec =  -10:0.5:0 
wvec = 0:0.5:5
# Zoom in cusp point parameter
#Ivec = -1.5:0.1:-0.3
#wvec = 0:0.01:0.15

nbr_I = length(Ivec)
nbr_w = length(wvec)

plt_I_w = plot()

@time for Ii in 1:nbr_I
    global I
    I = Ivec[Ii]
    
    for wi in 1:nbr_w
        global w
        w = wvec[wi]
        
        x_eq_for_fixed_w_I = Array{Any}(nothing,0)
        for x0 in x0s
            tmp =nlsolve(fp_BL!, jacobian_BL!,[x0])
            if converged(tmp)
               push!(x_eq_for_fixed_w_I, Float16.(tmp.zero[1])) 
            end
        end
        
        x_eq_for_fixed_w_I = unique(x_eq_for_fixed_w_I)
        
        # Extra cleanning when equilibrium at 0.0
        idx_cleaning = findall(x_eq_for_fixed_w_I .<= 1e-6)
        if length(idx_cleaning) > 0
            x_eq_for_fixed_w_I[idx_cleaning] .= Float16(0.0)
            x_eq_for_fixed_w_I = unique(x_eq_for_fixed_w_I)
        end

        nbr_FP = length(x_eq_for_fixed_w_I)
        if nbr_FP == 1
            plt_I_w! = scatter!([wvec[wi]], [Ivec[Ii]], c=:blue,
                                msc=:blue, ms=4)
        elseif nbr_FP == 2
            plt_I_w! = scatter!([wvec[wi]], [Ivec[Ii]], c=:violet,
                                msc=:violet, ms=4)
        elseif nbr_FP == 3
            plt_I_w! = scatter!([wvec[wi]], [Ivec[Ii]], c=:lightgray,
                                msc=:lightgray, ms=4)
        end
        
    end
end
plot!(xlabel=L"w", ylabel=L"I", legend=false, grid=false)

#### 3.5.2 Method 2

In [ ]:
function fp_generic!(Fct,x)
    global w, I
    Fct[1] = -x[1] + F(w*x[1] + I, a_E, theta_E)
end

In [ ]:
# Method 2 - (I,w) Stability diagram
gr(labelfontsize=13, legendfontsize=13,size=(600,400))

pars=default_pars_generic_target()
@unpack a_E, theta_E = pars

minF = F(-Inf, a_E, theta_E)
maxF = F(+Inf, a_E, theta_E)
x0s = range(minF, maxF, length=60)

Ivec=-20:0.5:10
wvec = -100:5.:300

# Zoom in on cusp point
#Ivec = -4:0.2:2
#wvec = 0:0.5:30

nbr_I = length(Ivec)
nbr_w = length(wvec)

plt_I_w = plot()

@time for Ii in 1:nbr_I
    global I
    I = Ivec[Ii]
    
    for wi in 1:nbr_w
        global w
        w = wvec[wi]
        
        x_eq_for_fixed_w_I = Vector{Float16}(undef,0)
        for x0 in x0s
            tmp = nlsolve(fp_generic!, [x0])
            if converged(tmp)
               append!(x_eq_for_fixed_w_I, Float16.(tmp.zero[1])) 
            end
        end
        x_eq_for_fixed_w_I = unique(x_eq_for_fixed_w_I)
        
        # extra cleanning when equilibrium at 0.0
        idx_cleaning = findall(-1e-6 .<= x_eq_for_fixed_w_I .<= 1e-6)
        if length(idx_cleaning) > 0
            x_eq_for_fixed_w_I[idx_cleaning] .= Float16(0.0)
            x_eq_for_fixed_w_I = unique(x_eq_for_fixed_w_I)
        end
                
        nbr_FP = length(x_eq_for_fixed_w_I)
        if nbr_FP == 1
            plt_I_w! = scatter!([wvec[wi]], [Ivec[Ii]], c=:blue,
                                msc=:blue, ms=4)
        elseif nbr_FP == 2
            plt_I_w! = scatter!([wvec[wi]], [Ivec[Ii]], c=:violet,
                                msc=:violet, ms=4)
        elseif nbr_FP == 3
            plt_I_w! = scatter!([wvec[wi]], [Ivec[Ii]], c=:lightgray,
                                msc=:lightgray, ms=4)
        end
        
    end
end
plot!(xlabel=L"w", ylabel=L"I", legend=false, grid=false)

#### 3.5.3 A note on stability of fixed points
Thus far, the stability of fixed points in the analyses using a geometric perspective and in the bifurcation analyses was assessed graphically only. However, having a quantitative measure of *how stable* is a fixed point, is very common and often desired. Since the fixed points can be computed numerically using a nonlinear solver, their stability can then be assessed by computing the **eigenvalues** of the **jacobian evaluated at a fixed point’s value**. In other words, for the 1D model, one needs to compute

$$ \lambda = \frac{d \dot{r}_T}{dr_T} \Biggr |_{r_T = r_T^*} = \frac{1}{\tau}(-1 + w \Phi'(w \cdot r_T^* + I))$$

and look at the sign to determine whether the fixed point is stable ($\mathcal{R}e\{\lambda\} < 0$) or unstable ($\mathcal{R}e\{\lambda\} > 0$). If $\mathcal{R}e\{\lambda\} = 0$ then the system is said to be marginally stable in the sense that there is no evolution of $r_T(t)$; it simply stays constant.

This mathematical computation, performed on the fixed points found numerically here above, confirms the monostable and bistable regimes.

In [ ]:
function eigenvalues_BL(fps, pars)
    #= ===============================================================
    Args:
    fps      : Fixed points rT* (1D array of float)
    pars     : Parameter structure to use
    
    Returns:
    eigenVal : eigenvalues of the linearized system
    =============================================================== =#
    
    # Set parameters
    @unpack w, I_ext_E, I_sel_E = pars
    @unpack tau_E = pars
    
    # Compute eigenvalues
    I = I_ext_E + I_sel_E
    eigenVal = (-1 .+ w*dPhi.(w*fps .+ I))./tau_E
    
    return eigenVal
end

In [ ]:
# Example: default values for w =J1-JI, I_ext_E=0.15, and I_sel_E = 0

I_sel_E = 0.0
pars = default_pars_BL_target()
@unpack J1, JI, r_spont = pars

w = J1-JI
I_spont = nlsolve(find_Ibias!, [0.0]).zero[1]

I = I_spont + I_sel_E

pars = default_pars_BL_target(w=w, I_sel_E=I_sel_E, I_ext_E=I_spont)
r_guess_vector = [0.05, r_spont, 3695.]
fps=[]
for r_guess in r_guess_vector
    tmp = nlsolve(fp_BL!, [r_guess])
    if converged(tmp)
        push!(fps, tmp.zero[1])
    end
end

eigenval = eigenvalues_BL(fps, pars)

for fpi in 1:length(fps)
  @printf("Fixed point at %f with Eigenvalue = %.3f\n", fps[fpi],
                                                        eigenval[fpi])
end

In [ ]:
function eigenvalues_generic(fps, pars)
    #= ===============================================================
    Args:
    fps      : Fixed points r*
    pars     : Parameter structure to use
    
    Returns:
    eigenVal : eigenvalues of the linearized system
    =============================================================== =#
    
    # Set parameters
    @unpack tau_E, a_E, theta_E, wEE, I_ext_E = pars
    
    # Compute eigenvalues
    eigenVal = (-1 .+ wEE*dF.(wEE*fps .+ I_ext_E, a_E, theta_E))./tau_E
    
    return eigenVal
end

In [ ]:
# Example: default values for w, I_ext_E (9. and 0. respectively)
# --> 3 FP

pars = default_pars_generic_target()
w = pars.wEE
I = pars.I_ext_E

r_guess_vector = [0.1, 0.2, 1.]
fps=[]
for r_guess in r_guess_vector
    tmp = nlsolve(fp_generic!, [r_guess])
    if converged(tmp)
        push!(fps, tmp.zero[1])
    end
end

eigenval = eigenvalues_generic(fps, pars)

for fpi in 1:length(fps)
  @printf("Fixed point at %f with Eigenvalue = %.3f\n", fps[fpi],
                                                        eigenval[fpi])
end

### 3.6 Cusp catastrophe surface

Stability diagrams are useful to investigate the effect of both parameters $w$ and $I$ together but unfortunately, they loss the information about the *values* of the corresponding *equilibria*. A last way to plot all the effects and results together is to plot the *cusp catastrophe surface*, that is a three-dimensional figure illustrating the values of the equilibria as a function of $w$ and $I$.

In [ ]:
using GLMakie
import GLMakie.plot as Mplot
import GLMakie.plot! as Mplot!
import GLMakie.scatter as Mscatter
import GLMakie.scatter! as Mscatter!
import GLMakie.contour as Mcontour
import GLMakie.contour! as Mcontour!
import GLMakie.heatmap as Mheatmap
import GLMakie.heatmap! as Mheatmap!
import GLMakie.text as Mtext
import GLMakie.px as Mpx

#### 3.6.1 Method 1

In [ ]:
#=
  MATLAB (version: 9.9.0.1495850 (R2020b)
  Update 1) code for 3D surface for Method 1

clear variables; close all; clc;

tau_m = 10e-3;
sigma = 0.5;
Phi = @(x) 1/(sqrt(pi)*tau_m) * (integral(@(z) exp(-x.*z.^2 - (sigma^4*z.^6/48)),-Inf,+Inf)).^(-1);

% Test
% Phi(0)

f = @(r,w,I) (-r + Phi(w .* r + I));

% Data matrix building
%[w,I,r] = meshgrid(linspace(0,5,51),linspace(-10,0,51),linspace(-0.1,2000,3001));
[w,I,r] = meshgrid(linspace(-1,2,31),linspace(-10,2,121),linspace(-0.1,1000,2001));

sz_V = size(w);
V = zeros(sz_V);
for x=1:sz_V(1)
    for y=1:sz_V(2)
        for z=1:sz_V(3)
            V(x,y,z) = f(r(x,y,z),w(x,y,z),I(x,y,z));
        end
    end
end

% Plot
fontsz = 18;

figure
p = patch(isosurface(w,I,r,V,0));
p.FaceColor = "#0BDB12";
p.FaceAlpha = 0.5;
p.EdgeColor = 'none';
view(3);
camlight
lighting gouraud

xlabel('$w$', 'interpreter', 'latex', 'fontsize',fontsz)
ylabel('$I$', 'interpreter', 'latex', 'fontsize',fontsz)
zlabel('$r_T^*$', 'interpreter', 'latex', 'fontsize',fontsz)
xlim([0,2]);
zlim([0,300]);
grid on

=#

#### 3.6.2 Method 2

In [ ]:
# Data matrix building

pars = default_pars_generic_target()
@unpack a_E, theta_E = pars

nbr_w = 300
nbr_I = 200
nbr_r = 200

wvec = range(-20, 150, length=nbr_w)
Ivec = range(-20, 30, length=nbr_I)
rvec = range(-0.05, 1, length=nbr_r)

V = [(-r + F.(w.*r + I, a_E, theta_E)) for w=wvec, I=Ivec, r=rvec];

In [ ]:
# Cusp catastrophe surface for Method 2
# (x,y,z) = (w, I, r_T^*)
fig = Figure(resolution=(600, 400), figure_padding = 25)
ax = Axis3(fig[1,1], elevation=0.2, azimuth=4.3, xlabel=L"w",
            ylabel=L"I", zlabel=L"r_T^*", xlabelsize=24, 
            ylabelsize=24, zlabelsize=28, xlabelvisible=true,
            ylabelvisible=true, zlabelvisible=true)

@time volume!(ax, wvec,Ivec,rvec, V, algorithm = :iso,
              isorange = 0.05, isovalue = 0.0, colormap= :greens)

fig

## 4 Application to an experimental-like stimulus

Thus far, the dynamics of the 1D model was investigated using a *constant* input current. However, different experimental tasks can be used to assess semantic priming. These tasks usually consist in the transient presentation of a prime word followed after a controlled delay by the transient presentation of a semantically (un)related target. The presentation of a word acts therefore as an external excitatory stimulus/input for the population of neurons coding for that word. This experimental protocol can then be modelled by two current **pulses** of width *width* and amplitude *ampli*. Moreover, the two pulses are separated by a time delay *delay*. The stimulus onset asynchrony (SOA) is then computed as $SOA = width_P + delay$. The parameters *ampli, width* and *delay* can be tuned for both the prime (P) and the target (T) words ($\rightarrow 5$ parameters in total). For a 1D model, a single pulse remains since only one word is coded.

This chapter aims at investigating the response of the model to an input current pulse rather than a constant input current. The goal is to understand what changes with respect to simulations with a constant input.

In [ ]:
function Ipulse(ampli, t_start, t_end)
    #= ===============================================================
        Define the time evolution of a current pulse.
    
    Args:
    ampli   : The pulse amplitude (float)
    t_start : The time instant (>= 0) at which the pulse starts [msec]
    t_end   : The time instant (>= 0) at which the pulse ends [msec]
    
    Returns:
    I       : The current pulse function
    =============================================================== =#
    
    I(t) = ampli * Float64(t_start <= t <= t_end)

    return I
end

### 4.1 Effect of the amplitude of the current pulse
#### 4.1.1 Method 1
(Found in the appendices of the master thesis)

In [ ]:
# Effect of the amplitude
w = 0.5 # Monostable version: w=0.02
T = 150.
pulse_start = 30.
pulse_end = 40. # Monostable regime: 60
r_init = 50.
I_bias= -30.
amplitudes = -3:0.5:0 # Monostable regime: -3:1:5

# For {I,r_T^*}-Bifurcation diagram
Ivec = range(-80, 5, length=200) # Monostable regime: range(-80, 100, length=200)
rvec = range(-10, 500, length=500)

gr(labelfontsize=16, legendfontsize=10, grid=false,
   size=(800,350), margin=10px)

######################################################################
# Time evolution of solution
plt1 = plot()
Xx = []
Yy = []

Xx_Ipulse = []
Yy_Ipulse = []

Xx_bif = []
Yy_bif = []

Xx_scatter = []
Yy_scatter = []

for amplii in 1:length(amplitudes)
    ampli = amplitudes[amplii]
    I_ext_ampli = -I_bias + ampli

    I = Ipulse(I_ext_ampli, pulse_start, pulse_end)
    pars = default_pars_BL_target(T=T, w=w, Iapp = t->I(t),
                                  rE_init=r_init, I_ext_E=I_bias)
    sol = simulate_BL_target(pars)
    
    push!(Xx, sol.t)
    push!(Yy, sol.u)
    
    push!(Xx_Ipulse, sol.t)
    push!(Yy_Ipulse, I.(sol.t))
        
    push!(Xx_bif, pars.Iapp.(sol.t).+pars.I_ext_E)
    push!(Yy_bif, sol.u)
    
    push!(Xx_scatter, pars.Iapp(pulse_end+1e-6)+pars.I_ext_E)
    push!(Yy_scatter, sol(pulse_end+1e-6))
end

plot!(Xx,Yy,color=cgrad(:lightrainbow),line_z = (-I_bias .+ amplitudes)',
      label=:none, lw=2,cbartitle = L"ampli",
      colorbar_titlefontsize=13)
plot!(ylabel=L"r_T(t)\ [\textrm{Hz}]")

# Time course of I pulse
plt2 = plot()
plot!(Xx_Ipulse,Yy_Ipulse,color=cgrad(:lightrainbow),
      line_z = (-I_bias .+ amplitudes)', label=:none, lw=2,
      cbartitle = L"ampli", colorbar_titlefontsize=13)
plot!(ylabel=L"I_{\textrm{app}}(t)")

l=@layout[
    a{0.7h}
    b{0.3h}
]
plt4 = plot!(plt1, plt2, xlabel=L"t\ [\textrm{msec}]",
            layout=l, grid=false, legend=false)

# Associated bif diag
pars = default_pars_BL_target()

rnull(I,r) = -r + Phi(w*r + I)
minmax = extrema([1e-6;amplitudes])
plt_bif = contour(Ivec, rvec, rnull, levels=[0], cbar=true, 
                  color=cgrad(:lightrainbow), ls=:dashdot,
                  lw=2)

plot!(Xx_bif,Yy_bif,color=cgrad(:lightrainbow),
      line_z = (amplitudes)', label=:none, lw=2,
      cbartitle = L"ampli", clims=minmax,
      colorbar_titlefontsize=13)
plot!(xlabel=L"I",ylabel=L"r_T^*\ [\textrm{Hz}]")

scatter!(Xx_scatter,Yy_scatter,color=cgrad(:lightrainbow),
         shape=:diamond,marker_z = amplitudes,
         label=:none, lw=2,cbartitle = L"ampli",
         colorbar_titlefontsize=13)
scatter!([I_bias],[r_init], c=:black,msc=:black, 
         label=:none)

plt5= plot(plt4, plt_bif, layout=(1,2), grid=false, cbar=false)

l=@layout[
    a{0.95w} b{0.05w}
]

tmp = -I_bias .+ amplitudes
cbar_len = length(tmp)
plot(plt5,heatmap(tmp.*ones(cbar_len,1), color=:lightrainbow, 
     legend=:none, xticks=:none, titlefontsize=16,
     yticks=(1:cbar_len, string.(tmp[1:cbar_len])),title=L"ampli"),
     layout=l)

#### 4.1.2 Method 2

In [ ]:
# Effect of the amplitude
w = 9. # Monostable regime: w=2
T = 60.
pulse_start = 20.
pulse_end = 30.#T
r_init = .2
I_bias= -2.
amplitudes = 0.:0.1:0.8 # Monostable regime: 0:0.5:3

# For {I,r_T^*}-Bifurcation diagram
Ivec = range(-5, 2, length=100) # Monostable regime: range(-3, 6, length=100)
rvec = range(-0.1, 1, length=100)

gr(labelfontsize=15, legendfontsize=10, grid=false,
   size=(700,350), margin=10px)

######################################################################
# Time evolution of solution
plt1 = plot()
Xx = []
Yy = []

Xx_Ipulse = []
Yy_Ipulse = []

Xx_bif = []
Yy_bif = []

Xx_scatter = []
Yy_scatter = []

for amplii in 1:length(amplitudes)
    ampli = amplitudes[amplii]
    I_ext_ampli = -I_bias + ampli

    I = Ipulse(I_ext_ampli, pulse_start, pulse_end)
    pars = default_pars_generic_target(T=T, wEE=w,
                                       Iapp = t->(I(t) + I_bias),
                                       rE_init=r_init)
    sol = simulate_generic_target(pars)
    
    push!(Xx, sol.t)
    push!(Yy, sol.u)
    
    push!(Xx_Ipulse, sol.t)
    push!(Yy_Ipulse, I.(sol.t))
        
    push!(Xx_bif, pars.Iapp.(sol.t))
    push!(Yy_bif, sol.u)
    
    push!(Xx_scatter, pars.Iapp(pulse_end+1e-6))
    push!(Yy_scatter, sol(pulse_end+1e-6))
end

plot!(Xx,Yy,color=cgrad(:plasma),line_z = (-I_bias .+ amplitudes)',
      label=:none, lw=2,cbartitle = L"ampli",
      colorbar_titlefontsize=13)
plot!(ylabel=L"r_T(t)\ [\textrm{Hz}]")

    
# Time course of I pulse
plt2 = plot()
plot!(Xx_Ipulse,Yy_Ipulse,color=cgrad(:plasma),
      line_z = (-I_bias .+ amplitudes)', label=:none, lw=2,
      cbartitle = L"ampli", colorbar_titlefontsize=13)
plot!(ylabel=L"I_{\textrm{app}}(t)")

l=@layout[
    a{0.7h}
    b{0.3h}
]
plt4 = plot!(plt1, plt2, xlabel=L"t\ [\textrm{msec}]",
            layout=l, grid=false, legend=false)

    
# Associated bif diag
pars = default_pars_generic_target()
@unpack a_E, theta_E = pars

rnull(I,r) = -r + F(w*r + I, a_E, theta_E)
minmax = extrema([-1e-6;amplitudes])
plt_bif = contour(Ivec, rvec, rnull, levels=[0], cbar=true, 
                  color=cgrad(:plasma),
                  lw=2)

plot!(Xx_bif,Yy_bif,color=cgrad(:plasma),
      line_z = (amplitudes)', label=:none, lw=2,
      cbartitle = L"ampli", clims=minmax,
      colorbar_titlefontsize=13)
plot!(xlabel=L"I",ylabel=L"r_T^*\ [\textrm{Hz}]")

scatter!(Xx_scatter,Yy_scatter,color=cgrad(:plasma),
         shape=:diamond,marker_z = amplitudes,
         label=:none, lw=2,cbartitle = L"ampli",
         colorbar_titlefontsize=13)
scatter!([I_bias],[r_init], c=:lightseagreen,msc=:lightseagreen, 
         label=:none)

    
plt5= plot(plt4, plt_bif, layout=(1,2), grid=false, cbar=false)

l=@layout[
    a{0.95w} b{0.05w}
]

tmp = -I_bias .+ amplitudes
cbar_len = length(tmp)
plot(plt5,heatmap(tmp.*ones(cbar_len,1), color=:plasma, 
     legend=:none, xticks=:none, titlefontsize=16,
     yticks=(1:cbar_len, string.(tmp[1:cbar_len])),title=L"ampli"),
     layout=l)

### 4.2 Effect of the pulse duration
#### 4.2.1 Method 1
(Found in the appendices of the master thesis)

In [ ]:
# Effect of pulse duration

w = 0.5 # Monostable regime: 0.02
T = 150.
pulse_start = 30.
r_init = 50
I_bias= -30.
durations = 0:2:16
I_ext_ampli = -I_bias -0.7 # Monostable regime:+5.

Ivec = range(-80, 5, length=200) # Monostable regime: range(-80, 100, length=200)
rvec = range(-10, 500, length=500)

gr(labelfontsize=16, legendfontsize=10, grid=false,
   size=(800,350), margin=10px)

######################################################################
plt1 = plot()
Xx = []
Yy = []

Xx_Ipulse = []
Yy_Ipulse = []

Xx_bif = []
Yy_bif = []

Xx_scatter = []
Yy_scatter = []

for durationi in 1:length(durations)
    duration = durations[durationi]
    pulse_end = pulse_start + duration
    I = Ipulse(I_ext_ampli, pulse_start, pulse_end)
    pars = default_pars_BL_target(T=T, w=w, Iapp = t->I(t) ,
                                  rE_init=r_init, I_ext_E=I_bias)
    sol = simulate_BL_target(pars)
    
    push!(Xx, sol.t)
    push!(Yy, sol.u)
    
    push!(Xx_Ipulse, sol.t)
    push!(Yy_Ipulse, I.(sol.t))
        
    push!(Xx_bif, pars.Iapp.(sol.t) .+ pars.I_ext_E)
    push!(Yy_bif, sol.u)
    
    push!(Xx_scatter, pars.Iapp(pulse_end+1e-6) + pars.I_ext_E)
    push!(Yy_scatter, sol(pulse_end+1e-6))
end

plot!(Xx,Yy,color=cgrad(:lightrainbow),line_z = [-1e-6;durations]',
      label=:none, lw=2,cbartitle = L"width",
      colorbar_titlefontsize=13)
plot!(ylabel=L"r_T(t)\ [\textrm{Hz}]")

# Time course of I pulse
plt2 = plot()
plot!(Xx_Ipulse,Yy_Ipulse,color=cgrad(:lightrainbow),
      line_z = [-1e-6;durations]', label=:none, lw=2,
      cbartitle = L"width", colorbar_titlefontsize=13)
plot!(ylabel=L"I_{\textrm{app}}(t)")

l=@layout[
    a{0.7h}
    b{0.3h}
]
plt4 = plot!(plt1, plt2, xlabel=L"t\ [\textrm{msec}]",
            layout=l, grid=false, legend=false)

# Associated bif diag
pars = default_pars_BL_target()

rnull(I,r) = -r + Phi(w*r + I)
minmax = extrema([-1e-6;durations])
plt_bif = contour(Ivec, rvec, rnull, levels=[0], cbar=true, 
                  color=cgrad(:lightrainbow), ls=:dashdot,
                  lw=2)

plot!(Xx_bif,Yy_bif,color=cgrad(:lightrainbow),
      line_z = durations', label=:none, lw=2,
      cbartitle = L"width", clims=minmax,
      colorbar_titlefontsize=13)
plot!(xlabel=L"I",ylabel=L"r_T^*\ [\textrm{Hz}]")

scatter!(Xx_scatter,Yy_scatter,color=cgrad(:lightrainbow),
         shape=:diamond,marker_z = durations,
         label=:none, lw=2,cbartitle = L"ampli",
         colorbar_titlefontsize=13)
scatter!([I_bias],[r_init], c=:black,msc=:black, 
         label=:none)

plt5= plot(plt4, plt_bif, layout=(1,2), grid=false, cbar=false)

l=@layout[
    a{0.95w} b{0.05w}
]

tmp = durations
cbar_len = length(tmp)
plot(plt5,heatmap(tmp.*ones(cbar_len,1), color=:lightrainbow, 
     legend=:none, xticks=:none, titlefontsize=16,
     yticks=(1:2:cbar_len, string.(tmp[1:2:cbar_len])),title=L"width"),
     layout=l)


#### 4.2.2 Method 2

In [ ]:
# Effect of pulse duration
w = 9. # Monostable regime: 2
T = 60.
pulse_start = 20.
r_init = .2
I_bias= -3.
durations = 0:1:15
I_ext_ampli = -I_bias + 0.7 # Monostable regime: 1.

Ivec = range(-5, 2, length=100) # Monostable regime: range(-5, 6, length=100)
rvec = range(-0.1, 1, length=100)

gr(labelfontsize=15, legendfontsize=10, grid=false,
   size=(700,350), margin=10px)

######################################################################
plt1 = plot()
Xx = []
Yy = []

Xx_Ipulse = []
Yy_Ipulse = []

Xx_bif = []
Yy_bif = []

Xx_scatter = []
Yy_scatter = []

for durationi in 1:length(durations)
    duration = durations[durationi]
    pulse_end = pulse_start + duration
    I = Ipulse(I_ext_ampli, pulse_start, pulse_end)
    pars = default_pars_generic_target(T=T, wEE=w,
                                       Iapp = t->(I(t) + I_bias),
                                       rE_init=r_init)
    sol = simulate_generic_target(pars)
    
    push!(Xx, sol.t)
    push!(Yy, sol.u)
    
    push!(Xx_Ipulse, sol.t)
    push!(Yy_Ipulse, I.(sol.t))
        
    push!(Xx_bif, pars.Iapp.(sol.t))
    push!(Yy_bif, sol.u)
    
    push!(Xx_scatter, pars.Iapp(pulse_end+1e-6))
    push!(Yy_scatter, sol(pulse_end+1e-6))
end

plot!(Xx,Yy,color=cgrad(:plasma),line_z = [-1e-6;durations]',
      label=:none, lw=2,cbartitle = L"width",
      colorbar_titlefontsize=13)
plot!(ylabel=L"r_T(t)\ [\textrm{Hz}]")

    
# Time course of I pulse
plt2 = plot()
plot!(Xx_Ipulse,Yy_Ipulse,color=cgrad(:plasma),
      line_z = [-1e-6;durations]', label=:none, lw=2,
      cbartitle = L"width", colorbar_titlefontsize=13)
plot!(ylabel=L"I_{\textrm{app}}(t)")

l=@layout[
    a{0.7h}
    b{0.3h}
]
plt4 = plot!(plt1, plt2, xlabel=L"t\ [\textrm{msec}]",
            layout=l, grid=false, legend=false)

    
# Associated bif diag
pars = default_pars_generic_target()
@unpack a_E, theta_E = pars

rnull(I,r) = -r + F(w*r + I, a_E, theta_E)
minmax = extrema([-1e-6;durations])
plt_bif = contour(Ivec, rvec, rnull, levels=[0], cbar=true, 
                  color=cgrad(:plasma),
                  lw=2)

plot!(Xx_bif,Yy_bif,color=cgrad(:plasma),
      line_z = durations', label=:none, lw=2,
      cbartitle = L"width", clims=minmax,
      colorbar_titlefontsize=13)
plot!(xlabel=L"I",ylabel=L"r_T^*\ [\textrm{Hz}]")

scatter!(Xx_scatter,Yy_scatter,color=cgrad(:plasma),
         shape=:diamond,marker_z = durations,
         label=:none, lw=2,cbartitle = L"ampli",
         colorbar_titlefontsize=13)
scatter!([I_bias],[r_init], c=:lightseagreen,msc=:lightseagreen, 
         label=:none)

plt5= plot(plt4, plt_bif, layout=(1,2), grid=false, cbar=false)

l=@layout[
    a{0.95w} b{0.05w}
]

tmp = durations
cbar_len = length(tmp)
plot(plt5,heatmap(tmp.*ones(cbar_len,1), color=:plasma, 
     legend=:none, xticks=:none, titlefontsize=16,
     yticks=(1:3:cbar_len, string.(tmp[1:3:cbar_len])),title=L"width"),
     layout=l)


### 4.3 Combined effects of pulse duration and amplitude

Method 2 

In [ ]:
# Combined effect of pulse amplitude and pulse duration

w = 9.
T = 45.
pulse_start = 20.
r_init = .2
I_bias= -2.
durations = range(0, T-pulse_start, length=4*Integer(T-pulse_start)+1)
amplitudes = range(I_bias, 4, length=50) # pulse ampli ∈ [0, -Ibias + 4]
I_thresh = 0.371 # = I_bias + I_ext_ampli = ampli
r_SN1 = 0.07 # DEPENDS ON THE CHOSEN w
gr(labelfontsize=15, legendfontsize=10, size=(600,400))


plt1 = plot()
@time for durationi in 1:length(durations)
    duration = durations[durationi]
    pulse_end = pulse_start + duration
    
    for amplii in 1:length(amplitudes)
        ampli = amplitudes[amplii]
        I_ext_ampli = -I_bias + ampli

        I = Ipulse(I_ext_ampli, pulse_start, pulse_end)
        pars = default_pars_generic_target(T=T, wEE=w,
                                           Iapp = t->(I(t) + I_bias),
                                           rE_init=r_init)
        sol = simulate_generic_target(pars)

        if sol.u[end] <= r_SN1
            scatter!([duration], [I_ext_ampli], shape=:circle,
                     c=:indianred, msc=:indianred, label="")
        else # Jump to high stable state
            scatter!([duration], [I_ext_ampli], shape=:circle,
                     c=:lightseagreen,msc=:lightseagreen, label="")
        end
    end
end

plot!(xlabel=L"\textrm{Pulse\ duration\ [msec]}",
      ylabel=string(L"\textrm{Pulse\ amplitude}","\n",
                   latexstring("(I_{bias}= $(I_bias))")))

### 4.4 A look at response times
Response or reaction times are often used in psychology experiments to measure cognitive abilities of participants. In the semantic priming paradigm, this response time is measured from the target onset until the population activity of a item reaches a response criterion.

Here below are the evolutions of the response time (RT) when the response criterion is the firing rate ($r_{SN,2}$) associated to the lowest saddle-node current ($I_{SN,2}$) or when the response criterion is set to $98\%$ of the final high steady state. 

In [ ]:
# Computation of response time for a fixed duration
w = 9.
I_thesh = 0.371 # Found using nlsolve, as with geometric perspective
r_SN1 = 0.07 # DEPENDS ON THE CHOSEN w
r_SN2 = 0.8631 # DEPENDS ON THE CHOSEN w

T = 45.
pulse_start = 20.
r_init = .2
I_bias= -2.

# High FP reached when percentage*FP value is reached
reach_high_FP = 0.98 
duration = 15.
pulse_end = pulse_start + duration
amplitudes = range(0, 4, length=41) # pulse ampli ∈ [-Ibias, -Ibias + 4]

######################################################################
RT = zeros(length(amplitudes));
RT .= NaN

for amplii in 1:length(amplitudes)
    ampli = amplitudes[amplii]
    I_ext_ampli = -I_bias + ampli

    I = Ipulse(I_ext_ampli, pulse_start, pulse_end)
    pars = default_pars_generic_target(T=T, wEE=w,
                                       Iapp = t->(I(t) + I_bias),
                                       rE_init=r_init)
    sol = simulate_generic_target(pars)
    
    # If I below I_SN1 (or r < r_SN1) then RT = +Inf because no jump 
    if sol.u[end] > r_SN1 # Jump to high state
        # The user can change the reference value to r_SN2
        idx_reach_high_FP = findfirst(sol.u .>= reach_high_FP*sol.u[end])
        RTval = sol.t[idx_reach_high_FP]-pulse_start
        RT[amplii] = RTval
    end
end

In [ ]:
gr(labelfontsize=11, legendfontsize=10, grid=false,size=(300,200))

plot(-I_bias .+ amplitudes, RT, lw=2, c=:black, label="")
plot!(xlabel=string(L"\textrm{Pulse\ amplitude}","\n",
                   latexstring("(I_{bias}= $(I_bias))")),
      ylabel=L"\textrm{Response\ Time}\ [msec]")
plot!(ylim=(0, floor(maximum(RT))))